In [ ]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv

from supabase_services import supabase_client
from composio import Composio, ComposioToolSet

load_dotenv()
supabase =supabase_client()
client = Composio()
toolset = ComposioToolSet()


In [ ]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """


# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


In [ ]:
""" checks if entity already exists """
existing_entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

### Outlook

In [ ]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App, Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, Composio, App
from dotenv import load_dotenv
load_dotenv()

client = Composio()
toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY, connected_account_params={"shop": "fdi1gh-xg"})

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [ ]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


In [8]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



### GMAIL

In [ ]:
""" OAUTH REGISTER NEW ACCOUNT"""
from composio import ComposioToolSet, App

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

request = entity.initiate_connection(App.GMAIL)

print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:

""" GET AUTH PARAMS FROM USER """
from composio import ComposioToolSet, App
toolset = ComposioToolSet(api_key="5z8t4g1e2xigxw6urlywf5")

integration = toolset.get_integration(id="f615f1bc-ebdd-4dcc-a0ec-47ce59a3c0bc")
# Collect auth params from your users
print(integration.expectedInputFields)

In [ ]:
connection_request = toolset.initiate_connection(
    integration_id=integration.id,
    entity_id="default",
)

# Redirect step require for OAuth Flow
print(connection_request.redirectUrl)
print(connection_request.connectedAccountId)

##### Gmail Fetch Emails

In [ ]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 
from pathlib import Path 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv", 
                                   output_dir=Path("./attachments"),
                                   output_in_file=False)

In [ ]:
tools = composio_toolset.get_tools(actions=[Action.GMAIL_FETCH_EMAILS])
today = datetime.now().strftime("%Y-%m-%d")

# Define your parameters explicitly
gmail_params = {
    "email_address": "michael@flowon.ai",
    "query": "after:2024/12/20 before:2025/01/01", ### YYYY/MM/DD
    "max_results": 500  # Max is 500 emails
}

task = f"fetch emails using these parameters: {gmail_params}"

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)
result

Fetch emails

User Id : string
key: user_id

Max Results : integer
key: max_results

Page Token : string
key: page_token

Query : string
key: query

Label Ids : array
key: label_ids

Include Spam Trash : boolean
key: include_spam_trash

GMAIL_GET_ATTACHMENT

Message Id* : string
key: message_id

Attachment Id* : string
key: attachment_id

File Name* : string
key: file_name

User Id : string
key: user_id

In [ ]:
tools = composio_toolset.get_tools(actions=[Action.GMAIL_GET_ATTACHMENT])

gmail_params = {
    "message_id": "193ebaa2d2426d29",
    "attachment_id": "attachment-bc0a91c4",
    "file_name": "Processed_FormW8_2024-12-21_23-11.pdf",
    "user_id": "michael@flowon.ai",}

task = f"fetch attachment using these parameters: {gmail_params}"
response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant."},
        {"role": "user", "content": task},],)

result = composio_toolset.handle_tool_calls(response)
result

In [54]:
base64_string = "JVBERi0xLjMKJeLjz9MKMSAwIG9iago8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0tpZHMgWyA0IDAgUiBdCj4-CmVuZG9iagoyIDAgb2JqCjw8Ci9Qcm9kdWNlciAocHlwZGYpCj4-CmVuZG9iagozIDAgb2JqCjw8Ci9UeXBlIC9DYXRhbG9nCi9QYWdlcyAxIDAgUgovT3V0bGluZXMgNDAgMCBSCi9BY3JvRm9ybSA3MSAwIFIKPj4KZW5kb2JqCjQgMCBvYmoKPDwKL0NvbnRlbnRzIDUgMCBSCi9Dcm9wQm94IFsgMC4wIDAuMCA2MTIgNzkyLjAwOCBdCi9NZWRpYUJveCBbIDAuMCAwLjAgNjEyIDc5Mi4wMDggXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9DMF8wIDYgMCBSCi9DMF8xIDE0IDAgUgovVDFfMCAyMiAwIFIKL1QxXzEgMjUgMCBSCi9UMV8yIDI4IDAgUgovVDFfMyAzMSAwIFIKL1QxXzQgMzQgMCBSCi9UMV81IDM3IDAgUgo-PgovUHJvY1NldCBbIC9QREYgL1RleHQgXQo-PgovUm90YXRlIDAKL1RhYnMgL1cKL1R5cGUgL1BhZ2UKL1BhcmVudCAxIDAgUgovQW5ub3RzIFsgNDEgMCBSIDQyIDAgUiA0MyAwIFIgNDQgMCBSIDQ1IDAgUiA0NiAwIFIgNDcgMCBSIDQ4IDAgUiA0OSAwIFIgNTAgMCBSIDUxIDAgUiA1NiAwIFIgNTcgMCBSIDU4IDAgUiA1OSAwIFIgNjAgMCBSIDYxIDAgUiA2MiAwIFIgNjMgMCBSIDY3IDAgUiA2OCAwIFIgNjkgMCBSIDcwIDAgUiBdCj4-CmVuZG9iago1IDAgb2JqCjw8Ci9GaWx0ZXIgL0ZsYXRlRGVjb2RlCi9MZW5ndGggNDMyNQo-PgpzdHJlYW0KSInsV2tv2zgW_W4g_4EfnUXMihSpx6Ao0Dw68ABJuo2LohgPFopNJ5y1JY8kJ-P59XtI2vJLkt2d3Z1-2AZoZIciL88999xzLwedN1dZWqq0JG_fvrm96l8Tj7x7d3l9RToeMT_5U-fNgP3DI4wMJp3QfhcSPyCh8CmTARnMOt0PWT4j5Hzwq13L3FoW2cX4JQPK_Z0XvvSiy5s788LNLY46CIKtg2g4m0vqebHdatj9pF4ouR-V2aPKCeEeZ8Pzauubmivy9e6_dZiLkfiShpIEcYB9QzKarW7_6Uc8aIfFrBMFNMbDtPPQ-XvnctAJqBBbQXlxFdS1mid5OTOHZhNSPisyyFVSLPKlg8kjPdzputPtI7I8TaYEl1DpQpEHlb_okbKrEHs9PP7hBRjnVJDQQC2jpgvgWD-gcnMDAy9fpYu7nfBRhNQPTMICGoYuYVcqL_VEj5JSmRsh4Uo_peShTMpFYb65VKnC3zWucv-aIhGTLCefU12qMTH_zH0YDSN78XGna95UBRkkv5Mvunx-zqZjnT6RJB0DinmG0_BpCHzG-kWPF8m0WCV1jcl7E1Ay2oAi6kBhyNiRrPpBSGNZ5bUecdmIuM-BeOPmDvFwG_Irb83owL4QEBZz6gkJWos1g956XsDerQrKd8tdPUVYHtM4ZljOqVwRDgkhi0KRxyXRG8gouUlLXWoAPVsUpV1ha9XVX--GrmtW7EYkGI6QOxF1XX1vou-hnoWPfFLBhclpa8wAWcYRCRlAC6Td8MeMlFmlGtK90H19faU6L-hT9vLGxPplrRRbu3Ytu3RalPliVOosLSxxTJ1NDa1K_A0rZon5G92Lu0I9EDTwkUGvCqn1BiGnEUor9DwaB8zdQL8onKoLE8_M3MaE8LrN5yerATmZJ0uVU3KdkTQrSaFMuG55_9PD8Iy2czs4pJ-IYsqOKlYIyT1C7bB-7-AEMQk8UJvtysmeWkvA5TNLJOHSfn97Se4ySpgUsscCzravflkj19F2M9jLCnQ3gCzz2MkU4L27H1ieb9KiJz-Q5lYTr3eXEQ2E0XcjT31wSyXji6pk2rZgXk276q32Q30EXJpNf-4OOZ6-ZguS5MoGmqRb5UrOeyGUokvPe9AW_v-H5gdhcKpUjJz_MvipKTds42eQCryHtruXiYR8pg-UjKCUf6jUVGuGwszdt3OVF1l6gTyNpgtX0yRXhR6bU5Lh2VSrvSQyn8ffAUanPPgiAI4AMm6DsPJLg7-toDvvcbTy7gbAx037z2z7H00TPTNolc-JkeNRNlMEFakmEwXJflHTJRllaYoPajw8s6Jp5RBfjiHq1jflyViZdDwuCp2qorCrdGrXrbyF8xE2eo9GSC-SzKp667pEIoZ0lUgEWDiLBUOBZAVoSn99Hr7_h5AzYmz7zVW_jSqVM-1V2TC_DtWvhjKvz5lhSK5GCsUE6oAzc5UWtovbju9SODzbSqL5zqg8qFBHDGRY_DkAfGGkJuK-b5j4pSda1UYclMrWjV34BI3dykjh5B8DAKLXCWYD0wDEd5Dsv1qUfCfu_duvW2DXTXKsxpevRjn4DuNc982Lh_HndW1hpDAfvV0Ds3YYlIuNyZAxg8kIrcm4y0r1w9qRemtH2p-Q5SrTVXsAIxPy4f3g6j0x86Ah-a-LXBdjbV0r5hunjsUF1t1mYzUlwzNG-j--311nxRF1oed5hi61HJ5fkBEmssRQHhNUMhpli3TH9ZIZymSJIqvmTSeLmDmxCXqVsgYUIe_FBN1dxT9SO67UoBC4cQCPDJNVHGNpZ1Jr2lhlWZmZlUj-tA0ucznBLwFcEZcMAhoFbtD4CKhIf-O3TOIPHDGrsa0u8zI0lvjQthqv2pT6hmqOGs9YAfFt7HJLgcTm-vCuYufyfUMcDV1M1tm43OjkvdXJfeoNu4VSO9OQ6Wzb88TBxeLGiwmvAbyti_l-RONd7CwFkrmNeX0K91qMO4yHNPPFqqbYQT3Zf3cJXAMw2PJXqy5hlP6gg7RThrOaW4fCTEgnXZtjUGRHKMN5_RnCUobLI5RBnWJQkh47iq3fNhRFnjlwC15uwNyH-MpoBroO8F1Z3-JZz49gKP4Uc06pOt6o6ZLVD7vHjqgFMDhCTp9XU6VfT86PaNxJakKvBJMk43FujCpqssyVKiHs85Ia51AsYEtImtEL8ylf5IkR-zxblGp4TqvkrCIxk2yalXb8TMhHek_JY_a7eVOnvRGaTA9ZWx1Gd3l_gGejUoqYA8-gHU_h-TQ8Ac9oG09vF88AbPTQUGNp8bxCCzNXKVG1F6QwRs18tH3JdB3St7OWIvMMf5zCB-L5FUYeWMyxap5rvEIB3xGy1sicMNp14tWZH1L_CFt9r-6MmHLLVi6PsPXkgvcr8apIKqQ5ZRtXV9CtbPDrBMqxIYj-rQKujbZNnlBdIhKVNon66rpN9NTa5Kqi9ISMNcbH3NxnkmczkjxmL2rd7Bpv3ChZQojjenIi_325nx-QXgReJSL_K9L7QTPpT7nvSaSvkRRHekMiLo-Q6HTSR_Wk38F1j_SHwcY1XOyJmMao_55vZtOubH5deDXCxqkvAwRk3v1MH6hx4fNkCQuid_1bupg94tth9-HhzqS7P-jfGfeujcv-baFz2PCVfxue7Ti49iyLGi8DgyGRZH7K4AP7vkfqmjOapeKUM7jkR3u98A-P4FFkD_jPmSUh2tQIdGIe9T1HpyBx8rMvSh-yXOknN26tcrw08lQluMmD199b1kTkMxpI0jPQMsus4LFlh6CGloxGjpQ_d6-e1eifhmYfwDhrJabqKZlOlxXvznsCDbJLz3uMw7b-Nx5-GfzUxuFKRepyLAOfSkyYJjnAI8IPhs2HJjgaRzU_3hkHa3lZS5pKNhiCauhjfiQhrG5yC-v72CdlOxa8oaMKmAJqtPOlFi5Z4zR4FB7cUVcFGEVHC1DW6IgpQLNjpeP6mytO8paK47EZUbB_6AxAVAfcte2PE_Ko8_IZcN3e9q6ve1_xrwG9bfDQA_3AjSN4hNbFIBLYM3HB7mPgbzJtfvKn7aiZIyd-ocEZrANOoyCykX9M8pL0-8cS12hA_JBvGrJhfviNs5JsnJXW999N4O6eAMNdDjfeXBOZETu3vJomemZyMYD2DXKVwMhcqlRNdFkc6OSwO0GbGz2DEJBFn8wXOQyNKkiWTpetxB9s0Wn_mq3DmilBjmsDvpWljFsNoaxxL9y4HqAmok1j2xSSxyk7rhiyfQSCNvuSVba3T0bKMGVJyuekxH-KPFpQRxruDybRmIkCw99quCyRgeZ2ICu1EiFljJs2wri0beRVl886tSfMVJKaroVkmo8wntlM2Z7W0qyCSnh6680ZdVuXjg2PqnxVyh3xOcWcOyYPxuAi_HTs7jdyNo22ncPqjBqnsTsRJtaeybyWLSrdwWuoJw9fmHce5srCmldBjbJ0rC3_aiiMtgnPPYWJe5yqIefhvzivtt22jSj4BfoHvhRwC5vgLi9L2kWB1k7QPCQIYgdFCwEFJa4kxqIokJRt9es7Z5c3USvKtv1gWCJ359xm5hz37LX1cKJicxoXSvJ0QvlQBr9UFyHrRA7ztVQQtnERL4t4uxrt1sBgkbTTcxk3NCtTdBeeb9fAG5prJmiXRJsyv23TXtMZ0RkJiNscW5cZH-e2x4JXwAuO4EHP-AE4ym_dgo3vRuvhEGyO0or0foiNS9cEszTSOS0pcPgO7vcH6CfVN1Q-GqVVvk7UDG2IzKixaIb3W_WAnihqj_G0GexKXVRH2Z0BbTveayxLEA1T5guHaAcrLNWVUmaP4hIGj4FkKHdMdsKgKI4dwV-fxSbYEJtiRB6xdpP78LJFmTSLxImeTgxWb1LrL41D1DHodNKfyExKyBR6YCYtuU6XKUbaIo2i5w11vT6j5sKwFenScREY-t2wasGTcLyCMr_CnYjXuhMMB3SfB4Edir47OWdPxEl7Mgjo7fZEnLQndfzvdCe9IG-ViIKpVbedMxLCtDX5dqjBCaoH9xUyOvv7JkEDbSW6sEql4u-tLH7siv2l9clKJFilkFrePlmrGFQjX2JQv0xqdUWbZQA2nQBaTr0LeaDPtCzmdcuWJO1WtrceN_nzWiZL3dkzdKsEo1QkKlAeeWnNcVJRyHl1WWtYtl3LStq4frGDTwbaxlTsTmJXgK-nk9NcKIRBiEUvRR70X6cI4ugP9JPZji_0GgqTE2d1LpL0KU12GEuVLwrJpJ_TC0wm3ol31Sov0v-kylOJkV5u2ql95WEwlogaO2_f61AGSZPnUiaKFp5X6XzVlYaCuVB9RDzskfFwwF0UTCHXyj5ohLuSRKArKY5K8vkuo0Rm-1KuF1bfCXutE74ZTX1o8kAaA7kuof3MmbSTNUG1S_TcJs56ksi082sw4-8mr6CL3-17u37jZgRcdAIcLsYfgXS9ClxzfZPQKr8-fWnYuU9UhIcHN13EKHJdWYpELhaYjvRJrvekGxv8g-iJ3VUPkJTAwam2wEjFiaRiznbTCYopy1ZgDjzsSELCVtQcDQt-4Y5gzTpY74REgADLDAnfV03xyt3sB85UdAIfr-c-3vQMLI6YTlqjr_3SWFB8EJRXBzVHUMrRQldovDjnpVWuiAMV-vrzcpVu9XfG0KcTglHjIlQ3CHgETSt_D78QiMQEIs5ou0A_xWtFGYsi17SDhG7KBRIyADidbHE75pLIt5fBvr_TmSylcvwW87wAu-3057HMeSfmg0dte5jng1bYml8913bhBxhWRdJeItk3jPPBumj1_DHtX1hIQP_rtKwOXHL9oDpC7z6bPbI8tjYe9dMbVsAb-mQkif64-DA3avT5DNF8pEEq8kdUUfVBrHc3NV_oBHwsX0DPm6UsL81qRAmFy5_IF5ltK8qQJB2qaxGX8AALJfd1nvoroj0SYutCAixKvdDgtB2mY_uo9TzDlWQ2Wj08VJxZvV8mWidRuIMWjpfgEFys0r8CXkxhVeTrS3TJXKZb-AiSqF1Z5cm-q66qLbliJY2NhmNwjxJEUnh8Z3PjPNaOzGlVFDySpOVsV5SK5rL4kSZ5T6JZnrt_cDuqUthWU_66T8hwLAvZmrLnFOqP-c5SUrmNfNaZq1vbdawk3oNiVcvX1klbSaQN6IiM-7ZtJglaqSAqHzZW5NZE9Tx6YGGJcgO9it1TL_0JDhpZS8POD9w6_zIdJfLJA9oaAkyvY0W-zWCpmcPR_HTyr47jBb-Nmv4w6kg-JYiR2kKiUJFOiF8sJPenjXTkdFtJeuCmm1FlnMOpR1Hb0Z9ab9p3zEoD4208T0EiZPX6Rq8eNMVnqIxmrenEwH3ja23EjvcQ5ghaRASShqwdLiIuE7Z7fqeNWqVsg8YW5ru0D3peV-S42mmNNNvdnp0d2N4mF3gP7X7obccDdo8D9lzWBhwMAoa6DZc5Y8DeMGBPYLKDSEes1eoOrI_APn--uru7-hs_0JJxsIYtsa6OR_0o3lmd4Kg6TmhhxfTgcAjo1wIWQCkq1YaSLYtxoOLUOusGwBk2OJn13GDlnt_DSjA7sgrVEaHaqCAEkV5oSbW-xuj857x4tL7JROuJ9Tum90tepXPoQQmGKyXsDKX6QHR0ph9OsEkUHgfAQ8f2RyJg9P0wgsGsu2ifCGVEGM2038aVDby2xX3HE_-cwRUZ2jU4g4sh831cXUs6NuMeLrbDIGpymg2lomNlD5zs4Y7eC39dhX98-DLwFs0FPqfnRf959Ps3-WTjtCvucFZPZx3u_wIMAOuJJTIKZW5kc3RyZWFtCmVuZG9iago2IDAgb2JqCjw8Ci9CYXNlRm9udCAvQ05TUVlKK1VuaXZlcnNhbFN0ZC1OZXdzd2l0aENvbW1QaQovRGVzY2VuZGFudEZvbnRzIDcgMCBSCi9FbmNvZGluZyAvSWRlbnRpdHktSAovU3VidHlwZSAvVHlwZTAKL1RvVW5pY29kZSAxMyAwIFIKL1R5cGUgL0ZvbnQKPj4KZW5kb2JqCjcgMCBvYmoKWyA4IDAgUiBdCmVuZG9iago4IDAgb2JqCjw8Ci9CYXNlRm9udCAvQ05TUVlKK1VuaXZlcnNhbFN0ZC1OZXdzd2l0aENvbW1QaQovQ0lEU3lzdGVtSW5mbyA5IDAgUgovRFcgMTAwMAovRm9udERlc2NyaXB0b3IgMTAgMCBSCi9TdWJ0eXBlIC9DSURGb250VHlwZTAKL1R5cGUgL0ZvbnQKPj4KZW5kb2JqCjkgMCBvYmoKPDwKL09yZGVyaW5nIChJZGVudGl0eSkKL1JlZ2lzdHJ5IChBZG9iZSkKL1N1cHBsZW1lbnQgMAo-PgplbmRvYmoKMTAgMCBvYmoKPDwKL0FzY2VudCA3NjQKL0NJRFNldCAxMSAwIFIKL0NhcEhlaWdodCA3MDgKL0Rlc2NlbnQgLTIyNwovRmxhZ3MgNAovRm9udEJCb3ggWyAtNyAtMjI3IDk4OSA3NjQgXQovRm9udEZhbWlseSAoVW5pdmVyc2FsXDA0MFN0ZFwwNDBOZXdzXDA0MHdcMDQwQ29tbVwwNDBQaSkKL0ZvbnRGaWxlMyAxMiAwIFIKL0ZvbnROYW1lIC9DTlNRWUorVW5pdmVyc2FsU3RkLU5ld3N3aXRoQ29tbVBpCi9Gb250U3RyZXRjaCAvTm9ybWFsCi9Gb250V2VpZ2h0IDQwMAovSXRhbGljQW5nbGUgMAovU3RlbVYgNzUKL1R5cGUgL0ZvbnREZXNjcmlwdG9yCi9YSGVpZ2h0IDMzMAo-PgplbmRvYmoKMTEgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL0xlbmd0aCAxNQo-PgpzdHJlYW0KSIlqYEAAB4AAAwAGzQDBCmVuZHN0cmVhbQplbmRvYmoKMTIgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL1N1YnR5cGUgL0NJREZvbnRUeXBlMEMKL0xlbmd0aCA0NjIKPj4Kc3RyZWFtCkiJfFHPaxNBFN5pkw3qErEQPKj1oa0KibVWhfZSCBHbioTa6MGLMNl53YzdH3Fm0rjQo4fIgogeihdP_hcl6MX_xYOed7IbqrM5eHQO3_u-4X2Pj_eIVZqzCCHXW-3O0xeP689DfohCUr-j2O02DuWQq14rCoJdXrTV9SV9OXEW9RWiL5b0olN7mw-z15MP5d_ONb18Xtcv6FvOsj52lqyyGWzZVtX6Qxxys8miLu4wDBVXcSvqx4J7PQV3N9bXGwY3HjRgbXV1DWbl3qwbOrFUGEjYCd1I9CNBFbIVaPo-7BVmCXsoURwWn_9yA5dAQQnKMKDiAKJ92EbO0O-i8FDAQzFwDwIq3R4PMYTmVgPwjesPpPH7MfjcxVAiA9UT0cDrwRMeRiruoyFdQUUMW0F3uwE0ZBDQGExKgR43OYUx8RBcFIqa-moguGTcVTwK5cqdR51nxZD7wHD_fzs2jxYwR0jb06NkrH-NicGl8XxS0qPJ7nRkp-zH7V8M_1UTn_aOeTfa2khph8rU_vljKebudG2_lYrWF6oanaUi4yR44zNZ_30S-2U6aPyKatUi7ue0VedG5_114UkmbxL7HyYVE7Onpz7K8AAQYvNpwplbmRzdHJlYW0KZW5kb2JqCjEzIDAgb2JqCjw8Ci9GaWx0ZXIgL0ZsYXRlRGVjb2RlCi9MZW5ndGggMjMwCj4-CnN0cmVhbQpIiVyQz2rDMAzG734KHdtDcRNYTyGwdQxy2B-a9QEcW8kMi2wU55C3n-KGDiqwQf6-n_gsfW5eG_IJ9BcH22KC3pNjnMLMFqHDwZMqSnDepq3Ltx1NVFrgdpkSjg31QVUV6IuIU-IFds8udLhX-pMdsqcBdtdzuwfdzjH-4oiU4Ah1DQ57GfRu4ocZEXTGDo0T3aflIMy_43uJCGXui1sYGxxO0VhkQwOq6ihVQ_UmVSsk96BvVNfbH8Or-1SIu3x6OWX39r5y8j24h7Izs-TJO8hB1gie8L6mGCIItR71J8AAqfJvsgplbmRzdHJlYW0KZW5kb2JqCjE0IDAgb2JqCjw8Ci9CYXNlRm9udCAvRUFWSEFGK0Fkb2JlUGlTdGQKL0Rlc2NlbmRhbnRGb250cyAxNSAwIFIKL0VuY29kaW5nIC9JZGVudGl0eS1ICi9TdWJ0eXBlIC9UeXBlMAovVG9Vbmljb2RlIDIxIDAgUgovVHlwZSAvRm9udAo-PgplbmRvYmoKMTUgMCBvYmoKWyAxNiAwIFIgXQplbmRvYmoKMTYgMCBvYmoKPDwKL0Jhc2VGb250IC9FQVZIQUYrQWRvYmVQaVN0ZAovQ0lEU3lzdGVtSW5mbyAxNyAwIFIKL0RXIDEwMDAKL0ZvbnREZXNjcmlwdG9yIDE4IDAgUgovU3VidHlwZSAvQ0lERm9udFR5cGUwCi9UeXBlIC9Gb250Ci9XIFsgNzAgWyA4OTIgXSBdCj4-CmVuZG9iagoxNyAwIG9iago8PAovT3JkZXJpbmcgKElkZW50aXR5KQovUmVnaXN0cnkgKEFkb2JlKQovU3VwcGxlbWVudCAwCj4-CmVuZG9iagoxOCAwIG9iago8PAovQXNjZW50IDgyNgovQ0lEU2V0IDE5IDAgUgovQ2FwSGVpZ2h0IDcyNQovRGVzY2VudCAtMTk2Ci9GbGFncyA0Ci9Gb250QkJveCBbIDAgLTE5NiAxMjc2IDgyNiBdCi9Gb250RmFtaWx5IChBZG9iZVwwNDBQaVwwNDBTdGQpCi9Gb250RmlsZTMgMjAgMCBSCi9Gb250TmFtZSAvRUFWSEFGK0Fkb2JlUGlTdGQKL0ZvbnRTdHJldGNoIC9Ob3JtYWwKL0ZvbnRXZWlnaHQgNDAwCi9JdGFsaWNBbmdsZSAwCi9TdGVtViA3NQovVHlwZSAvRm9udERlc2NyaXB0b3IKL1hIZWlnaHQgNTAwCj4-CmVuZG9iagoxOSAwIG9iago8PAovRmlsdGVyIC9GbGF0ZURlY29kZQovTGVuZ3RoIDE1Cj4-CnN0cmVhbQpIiWpggAAmgAADAASLAIMKZW5kc3RyZWFtCmVuZG9iagoyMCAwIG9iago8PAovRmlsdGVyIC9GbGF0ZURlY29kZQovU3VidHlwZSAvQ0lERm9udFR5cGUwQwovTGVuZ3RoIDM3NQo-PgpzdHJlYW0KSIl8kcFLAkEUxp3UjWqxEowgs0dYIIWKpw4hSCR5CcWKTsHmjDqkuzI7Cf4HguChLh07-B903urSMYk6di46dN_ZdoN29-SpOTze7_vmfQ9mUCA0FUAIRfcLJweF4lYBa-ekTKsce-q2WBHxgZwQq0gsh0RCjg3s03jIscbhV3ndfJ433xbNF3lD7MrJQBghKTJ-AjSpionPLentbpMdpocshlsznwPaj2dE7aOpTUmsY6GlM4wWkotFrgX9WBEZ2wri_6A1QHQnmTMFBcr0HdcUYwcKZg0lbYBWieM4H1fzYBVcHNgmOVelTlrqiDouKMm6L5W2rapcoZJXo6U6we9ToEdgCT-sTjuKfolSmEDrHoDwzxbSC3Jo3gICT6VtnpS-bQforZefM6bH5JNtiPMdMD52rakc783szbLkviIeZ1tkeRn4XbLrqz7oPW55KTsipO6rcSjni_MSPW5M0bUYqykUVH9nAkGbPG3J8AAwDrHrC1CmVuZHN0cmVhbQplbmRvYmoKMjEgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL0xlbmd0aCAyMzEKPj4Kc3RyZWFtCkiJXJDBasMwDIbvfgod20NxG7qdQmBrKeTQbizbAzi2khka2SjOIW8_xQsdTGCD_P-f-C19qs81-QT6nYNtMEHnyTGOYWKL0GLvSR0KcN6mtcu3HUxUWuBmHhMONXVBlSXoDxHHxDNsXlxocav0GztkTz1svk7NFnQzxXjHASnBHqoKHHYy6GrizQwIOmO72onu07wT5s_xOUeEIveH3zA2OByjsciGelTlXqqC8iJVKST3T1-ptrPfhhf38VncxdNrkd3r-8LJ9-ARyk7MkifvIAdZInjCx5piiCDUctSPAAMAqbpvsQplbmRzdHJlYW0KZW5kb2JqCjIyIDAgb2JqCjw8Ci9CYXNlRm9udCAvQ0ZVVllKK0hlbHZldGljYU5ldWVMVFN0ZC1Sb21hbgovRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZwovRmlyc3RDaGFyIDAKL0ZvbnREZXNjcmlwdG9yIDIzIDAgUgovTGFzdENoYXIgMjU1Ci9TdWJ0eXBlIC9UeXBlMQovVHlwZSAvRm9udAovV2lkdGhzIFsgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAyNzggMjU5IDQyNiA1NTYgNTU2IDEwMDAgNjMwIDI3OCAyNTkgMjU5IDM1MiA2MDAgMjc4IDM4OSAyNzggMzMzIDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiAyNzggMjc4IDYwMCA2MDAgNjAwIDU1NiA4MDAgNjQ4IDY4NSA3MjIgNzA0IDYxMSA1NzQgNzU5IDcyMiAyNTkgNTE5IDY2NyA1NTYgODcxIDcyMiA3NjAgNjQ4IDc2MCA2ODUgNjQ4IDU3NCA3MjIgNjExIDkyNiA2MTEgNjQ4IDYxMSAyNTkgMzMzIDI1OSA2MDAgNTAwIDIyMiA1MzcgNTkzIDUzNyA1OTMgNTM3IDI5NiA1NzQgNTU2IDIyMiAyMjIgNTE5IDIyMiA4NTMgNTU2IDU3NCA1OTMgNTkzIDMzMyA1MDAgMzE1IDU1NiA1MDAgNzU4IDUxOCA1MDAgNDgwIDMzMyAyMjIgMzMzIDYwMCA1MDAgNTU2IDUwMCAyNzggNTU2IDQyNiAxMDAwIDU1NiA1NTYgMjIyIDExNDggNjQ4IDI1OSAxMDc0IDUwMCA2MTEgNTAwIDUwMCAyNzggMjc4IDQyNiA0MjYgNTAwIDUwMCAxMDAwIDIyMiA5OTAgNTAwIDI1OSA4OTEgNTAwIDQ4MCA2NDggMjc4IDI1OSA1NTYgNTU2IDU1NiA1NTYgMjIyIDU1NiAyMjIgODAwIDM3OCA0NjMgNjAwIDM4OSA4MDAgMjIyIDQwMCA2MDAgNTAwIDUwMCAyMjIgNTU2IDYwMCAyNzggMjIyIDUwMCAzODQgNDYzIDgzNCA4MzQgODM0IDU1NiA2NDggNjQ4IDY0OCA2NDggNjQ4IDY0OCA5MjYgNzIyIDYxMSA2MTEgNjExIDYxMSAyNTkgMjU5IDI1OSAyNTkgNzA0IDcyMiA3NjAgNzYwIDc2MCA3NjAgNzYwIDYwMCA3NjAgNzIyIDcyMiA3MjIgNzIyIDY0OCA2NDggNTU2IDUzNyA1MzcgNTM3IDUzNyA1MzcgNTM3IDg3MCA1MzcgNTM3IDUzNyA1MzcgNTM3IDIyMiAyMjIgMjIyIDIyMiA1NzQgNTU2IDU3NCA1NzQgNTc0IDU3NCA1NzQgNjAwIDU3NCA1NTYgNTU2IDU1NiA1NTYgNTAwIDU5MyA1MDAgXQo-PgplbmRvYmoKMjMgMCBvYmoKPDwKL0FzY2VudCA5NTIKL0NhcEhlaWdodCA3MTQKL0NoYXJTZXQgKFwwNTdzcGFjZVwwNTdwZXJjZW50XDA1N3BhcmVubGVmdFwwNTdwYXJlbnJpZ2h0XDA1N2NvbW1hXDA1N2h5cGhlblwwNTdwZXJpb2RcMDU3emVyb1wwNTdvbmVcMDU3dHdvXDA1N3RocmVlXDA1N2ZvdXJcMDU3Zml2ZVwwNTdzaXhcMDU3c2V2ZW5cMDU3ZWlnaHRcMDU3bmluZVwwNTdjb2xvblwwNTdzZW1pY29sb25cMDU3QVwwNTdCXDA1N0NcMDU3RFwwNTdFXDA1N0ZcMDU3R1wwNTdJXDA1N01cMDU3TlwwNTdPXDA1N1BcMDU3UlwwNTdTXDA1N1RcMDU3VVwwNTdXXDA1N1lcMDU3WlwwNTdhXDA1N2JcMDU3Y1wwNTdkXDA1N2VcMDU3ZlwwNTdnXDA1N2hcMDU3aVwwNTdqXDA1N2tcMDU3bFwwNTdtXDA1N25cMDU3b1wwNTdwXDA1N3FcMDU3clwwNTdzXDA1N3RcMDU3dVwwNTd2XDA1N3dcMDU3eFwwNTd5XDA1N3pcMDU3cXVvdGVyaWdodFwwNTdidWxsZXRcMDU3ZW1kYXNoKQovRGVzY2VudCAtMjE0Ci9GbGFncyAzMgovRm9udEJCb3ggWyAtMTY2IC0yMTQgMTA3NiA5NTIgXQovRm9udEZhbWlseSAoSGVsdmV0aWNhTmV1ZUxUXDA0MFN0ZCkKL0ZvbnRGaWxlMyAyNCAwIFIKL0ZvbnROYW1lIC9DRlVWWUorSGVsdmV0aWNhTmV1ZUxUU3RkLVJvbWFuCi9Gb250U3RyZXRjaCAvTm9ybWFsCi9Gb250V2VpZ2h0IDQwMAovSXRhbGljQW5nbGUgMAovU3RlbVYgODQKL1R5cGUgL0ZvbnREZXNjcmlwdG9yCi9YSGVpZ2h0IDUxNwo-PgplbmRvYmoKMjQgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL1N1YnR5cGUgL1R5cGUxQwovTGVuZ3RoIDQxOTMKPj4Kc3RyZWFtCkiJlFV7VBTnFZ9h2NnltTwmg8ri7hQhyFPkDSoBF4EgrwgmoBZZ2UVXYCHLS4yvUKMgYKOetuQkWtNUPanaBCnBWNFC4jMlKMQ1OKiLZtWzQTGK5w7ebU5nadMc2796zpzvu99893fnd3_fvd-QhKMDQZIkp05b_npRZkiGrqJeV6sv1eTo6nRZBfm12rBlVZUag91HJShIYaaj4OvKYiKuf37xeZYEHroLMz2P-DKrvQgHMRIBpKO6qrrRqF-7rpabnxAfHyqOCRHTY1QoFxkREclNT1Ep2qo1Oi6_saZWV1nDvWoorTJWVxk1tTptOJdSUcEts4eo4ZbpanTGevvL_3Dj9DWchiswarS6So2xnKsqE_f0Wl3FGp1xrc7IpRrrSssrNTWl6_QGnYFLSed0G0or6mr09bqKRq5CX6oz1Oi0XO06Y1Xd2nVclt5QVdtYrRONNUaNsZFLr1yTEcppDFquUtPIiSSNurV6kaZRBOkNXKnOWKsR5_V1Rn2NVl9aq68y1ITPS8svsAeJ5rS6MkJUgiQImiBkDoSbA-HpQvjRRDBBhFNEFEXEE0QiSSyeRdQSRAtBpNqFyybyiDGSJpPIerKDPE72Obg4BDs0OFipWZSRGnP0dNzj-KPET7JcsluCdB7dTnfSV6XR0nTpO9JemYcsR3bUiXNa4FTr9KXThPNLznudH7pEuex2ue86x3W_67DrD25z3Va6GdzOu_0oT5UfkD93D3Rvcz_l_szDz0Pv8bHHU8-ZnvGeudGtvcJ4LymO_r1Uq6PQPJVna6ZhN55jMQn2SeABjRz-jQX7wrZXaqOLp21IQnFNC2dZu4X2lRwvyYX5W_thf7_BPnQ0eN03QfK_H6Znm3ALclgk_jIf3EDWM_4UyLX30APdtSGc0tTOXugYu2b6TUxe9ta4V5Kb7pxTMte32QZg6P-CfLLtoO0wO0YLx6c2SWJpkZOwqnccKPIWUNQtYYI9T_9yS2Gxskj67bHBGyqgkJLKcRSX1D-PaSBPWSj4EFewoRBWBoGw0AdiP4UgKwQrM-kA9C3ARRjhg-qLOOd7DFb-jAO5heoXcT-Abz8sgggfUOfAnJBpnBXDPsVAXOiDsWUYFDqNu9T6EyPv83R7-aai4k0ykdGHxy6YfB5PM1oBTTzZC91ULzSx0M1jN_0CDCg7dVq-1SQMmshuC5y0UN3wmMWI97AEjGB8D0ogAuZvhhI0onEzluB8paWNnWyDDAiAgDbM8PvFLszAAAzYBRnPlPKD9cK7gyR8ZKZAmCpiM2luQ9SbwYp0yLK9a5bKW03CFyayxwxHRI1YoZg1N3ffe6QwXSpMCspZja4pqsDs2NzNuFAmlJpsBP3B3Qcnv-uT3e2_fRk8fWBm0LB4bsGxHPrvVYqs4dAgbDR5fWxeYoYOS66FeQIGGGPb39_f9oHi9terImNSC-ODC7-636yKpfHlZmss-CtgFlDXwM-sGYw5oGqXMuN9f-zpu-QDM8LPozv6qcNQsUNppm-1dQ09UAz3laYkZWnVW7e2tm1RielBoomE-zwlxInppdN43aaRJMBXmMhD4vemECkOecM2mJC044RdV7hpgnkm8s9mOGuhhPVT2Sw-oV-1OX6zU9J17OThPoX17xlhKEkrSIzN6_9mh53mS7snksFdAQm3QAauEOs_il4rX39rXYWqpb0TSiRwYjrwkAnSTF6dZugxL7Ewk3BC8GTHP18cFVe4eF748qt3vzs7ZFUxAughkAXp24sH0EmBxXmYjW9g0HEsgDKViEo_B4onMCPxEfom5VZllorfkAD56z9ANAQoGOHrHs3Sffa0hX_wJPhZKWGPmIJt9QJBRmMQRqIUq97C7TIr3SGWiww6YA74yaZPxtRUD3tMXl2WTAvstGRbmBHwhlXszYG-2_zAa0kLc_Nio7LOmpVMdyaMsku78s7XKJnPF1YVZ8b5YMBkKvjCjKc3YPbdNZdiDyuZU-cOdZ277GPJvYfzRAySc5eik4oxWZrZoS86B6_06lLVRZqMJfmfjExXRppdIa_jlmQzfDbGTAphQgUbl6-OTHhteORO5-hj65nkaCUzjjbsY1H-W3MOuCgg-RrMhhY4iNQQzhTlwdkhYTgbOT4Y_K9c7rxwVNXeIsGlOwoj5iiY8fT8E1feUf2roeCv_91UQAnBLIoWUPT-LzQr8cv7DLJ7A37AuSiWGvlPIyNtTZ4bZ8KYb7d7v2Q5qFcAv30CJZLJmk8OlUs8aOZB70xbDsNLjgmwR20_GErD6389nrYx3uNWMFqzbQyNyAFeBYqad70ZviiN4oSVNhEhy5g4SENfqaJSRVz6u7Kgaj9SrEfTg8c7vryus_VsjOrjigPlcS3Zinwd7RVdF5G37y6ea6KGU1erUVCictoOSg28oKZJz-zCsut1FScsJOF0Dp05zFIgdXoLN51pagFAuPF-sgQNSAgbb8qlEbnJnUUeirwFbV40HEQNAwcJD0aLXz59yo5OLXywhmeHLEKYsmNeEMLDU2ToIBfwUZ0BwesVmHLNKNoGtSX_cUfx_uobsxUYSQt_2g6fWiww0WssJ23BUCrIOVtUmgQ_Hjb27R8i6gt_7OPFXgY5XFU3HfmbXoxq-SNvKYeTvNwhPcSUzOYmdPMpilP7wuCKw4sgA00cxq990T5Y6ACo2LEDIIgbBiCIeXZnRVIHFBZaVhQjR5XMUKB9UiLN6QWc4DBxaC_bfmTqV-smtGD9VPFDeQ-a7yqCjOJO449te4KqzT9LjLuD0aB4LKEY5FdLyAgBxKvB7xQAweifI0uiYLiL7AIyaKBEXWCyTgGV13XwBJVtz12qj7CGKAoMMxEkdlEOYhumisnlTn7VbPJH_kj_jm6FfdXd9X9av6VdXX5dB26R3OtADatboQWFCm3AMjdVa7fNouZFv1drmHYvue0sPdpYvcwxUpPXZnGuqZUIeZSj-We7nSKJIFIIBWabcAuMpZdrt8u_tQmq73s6EDXJLES8PV1q4Z860ECaUK20FvDJSbuP8SC_KHcEah5zv0NaAU8QU4DEesujDU8xSIAXpiln0ofuYIjn0YLpN3kZD9G2chCY0RdMlGqOj6aZJcqVunRXKVHgp_1wI18G7lGq7263wO6rjMzvxFQlvuWKYzF5cXBERsSx13CdGTKb8Je0bVihyaf_PoYUoWRShCPL47po3YyfMej_UiBcUUXTADdjOHpXPjz1gLI3dPvOzdz2-Z6sv7u34qGVHS0595BGPIv7-iZpv7_sAi7yNPhJ2E6tczuqq7XMcYLcTM24L7XKHXCRCyEeobUY_A-pRTKKGNgb4DGBbjEI3_QPLmehjCtj8Rxzy6hUYKYUwCqVhBlVJPUyshwAYh6wW2SlJaEf-FLLDD316rWesj5tW-h6U1HRzelL93EHO7KBge1JbVprMkO_mj9OYpbnjkL2JP2OdaWKAkspQlxCHI1EX0ww6SU6lWzwuBE0EzIW4QQfES57wqBBG-mfK_cAo2od-8BGW_OCnajPhMPpSuhQwDKV0MTEMd0vjSJrEIOnHNG6ACetgKiUOpdYUpZ9kNWZqOsJZssKH1sEhDNdiL2ZAL4ccgzPKNA7rKI68yjCVncfkRu0xvdxoVZYz-Q35MmdVKsxyEK_MVy5znktppjBYdFV2as07bUKv7A3xYuqKWBz25vLKaw3_qrx2QLp-4OYnfz_hARIuF3fmfbh7h2Fz4ae1RqjgYZTf0SAXmP5p6P2x0cba9l491mr4d31uVJJ5X-BWo9AblhO-BYf7mNrT-1obTn71T2nR6ffOf1ZRvL9M8tyXa5nbQPvDvCzdNzZosgsX5XfgiOi3xh89cPiKvv_2n38MDIZeCpwkCdsVEx4RbUTq5iI-uaYt66oBQjvAC_zAbzroMOz1hC0LNxkLoJJmQbdjmipq54XwTASv3MCv1NYaaUZPDMagezgUQkHfaQFdhTGczchbtnyqAYe81ed4XtMPXi11axMPG39hYJUdmqjAyMcJ9gisKGD1a2dVphgwdAbNI37o14k6CPv21snrp41FTNgea6MarZigRHx6boB67vALgb6TVk6kKjN0aV-pBoIMy3gp9oIl8jMqRApxm9ftj7WgNwSq32PUR7DYHTxIpiFSQYc70uawVQ6izuQwVQDeHR9Dey4UfECk9i57-o33YZ79YnT9lLJOk9zQLIF9JYPs3QDFNMBvU1OfmJ5tZXVnTj8RdmnBR-XUczu8EJv8bb392wzBC1dOcU4M3z2A97zA3y7CcZaZLFh7k9J8aWVutLJZhGHrX_WO3C-GzQvagNfm5RuwmFGwQH58mLRzITOqN389Q0LziQacM4bOJkgHt-OoyCoselsA8GxnqHHSjNqJcGyKOrY1zvV-YkrhmFr-g0wpf0-THfNLstUmDU3bXDQpr2pt0Ed673UPNhYu2quhIM26KXGm96BY-dF_2neJqkATlGUFQ3plWYB90zTbNM2k6OlEcwGS8WICTb1KS6zOL1_fgxBcqkYodxiKXlb387NKyzKlaL5XWVlu0sNlou1j4zOo8NcanPca27M-mlNWMKsdyAWj3NAbNsMXZAO8-mcxjAPMznZHMG77FB56VZwppFpUKG6pOJIHuWqX--xwkXVLws7V937tO3o8ngJH5HIW69e6fqufEmSKspe_Iu4f5gSkjMWr5MaMhecnGt4fcHGBWuNBUy422bm3UDlZulcOAntLwEqasHGpHck0qvrIr0PfiYEWOzaKjgo4uQHOAIiIfIBjIDJEBBOo0kkRobjCAyQ7HrQtbZ0d7fGog5Hx8SGhsa0wGhiLB2CIJlWUh1zU_bur1NWMSnpog0qi9hL6MqE_haC8Bcru7l2V-iXj6sxI9fjCviX8A2P2phikpf8CtewhFBLshNq9TahuZ4gy2Ew5lIH_EGy88F7Fq5aFelBdlSDhoAKd5fEs7YoO-TY4u3Ue7rPwhQRvYsfp4KH4TnwteD1ZMrpiAqj0D69IuRAvc-V_5RevnYjM2EfnUhAt-O1KmSGENNbE_xtm-5lU2-yZd_LS_BJjMlNiU040baLLDpJZyytRdNPaZnoDKeCEhw2-yF2sYDPtz6UGi1rKElLzhnhzqDFxNOgnK6m5E8xr5PzVFCwjgUtjTMlrv6iQYLBCOzlUfvlDBh758Zfb1ZJVIKyp1rJm8L7iS7dUnmDsFnOIc1Q8hTHKzFqt_GVY7gABouVUq6HcnZWYR-0fE-dqty5Tqh2bqW3w4i7ill9d4hs5oLYBLcwnIRABtNI8RETmp_JwdSlnivBnJ15xtFocCHzdCbkZ-n2y7uEuv36QQYmyAATZtC5hWEq_A3iIJ_zZShiAv4WEjjUMKGd5pcCzjMcV1vSMp0rs3R_kbOFape71Mp93TtPpp0nM0hXDq4L4zZuy85ZZ8jJKSneZhQ2z-CF6h0VlQWVhrbPq5-eM3qOc6N8TI5Xe2C8Bb9j0Cx7czcUv9nUI7FRGUXbkSlOTSs4tfQT_TvNAwNW68CAudPff_p0f8lTDqFBQAN6Z7aW5oh80Zlt_TGbeeaUO9PKMeMQg1MlT0qUvkO85Te2EfJVb_me-H8BBgBb-8u2CmVuZHN0cmVhbQplbmRvYmoKMjUgMCBvYmoKPDwKL0Jhc2VGb250IC9EV0dCTUgrSGVsdmV0aWNhTmV1ZUxUU3RkLUJsa0NuCi9FbmNvZGluZyAvV2luQW5zaUVuY29kaW5nCi9GaXJzdENoYXIgMAovRm9udERlc2NyaXB0b3IgMjYgMCBSCi9MYXN0Q2hhciAyNTUKL1N1YnR5cGUgL1R5cGUxCi9UeXBlIC9Gb250Ci9XaWR0aHMgWyA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDI2MCAzMTQgNDgxIDUyMCA1MjAgNzc4IDYxMSAyNjAgMzE0IDMxNCA0MDYgNjAwIDI2MCAzNjggMjYwIDMzMyA1MjAgNTIwIDUyMCA1MjAgNTIwIDUyMCA1MjAgNTIwIDUyMCA1MjAgMjYwIDI2MCA2MDAgNjAwIDYwMCA1MDAgODAwIDU1NiA1NzQgNTM3IDU3NCA1MDAgNDgxIDU1NiA1NzQgMjYwIDQ4MSA1NTYgNDYzIDc1OCA1OTIgNTU2IDUzNyA1NTYgNTc0IDUzNyA0NjIgNTU2IDUyMCA3NzggNTM2IDUyMCA0ODEgMzE0IDMzMyAzMTQgNjAwIDUwMCAyNjAgNTAwIDUyMCA0ODEgNTIwIDQ4MSAzMTUgNTIwIDUyMCAyNTggMjU4IDUwMCAyNTggNzc4IDUyMCA1MDAgNTIwIDUyMCAzNTIgNDYzIDMxNSA1MjAgNDYzIDc0MCA0ODEgNDYzIDQ0NCAzMzMgMjIyIDMzMyA2MDAgNTAwIDUyMCA1MDAgMjYwIDUyMCA0ODEgMTAwMCA1MjAgNTIwIDI2MCAxMTExIDUzNyAyNDAgNzU5IDUwMCA0ODEgNTAwIDUwMCAyNjAgMjYwIDQ4MSA0ODEgNTAwIDUwMCAxMDAwIDI2MCAxMDAwIDQ2MyAyNDAgNzQwIDUwMCA0NDQgNTIwIDI2MCAzMTQgNTIwIDUyMCA1MjAgNTIwIDIyMiA1MjAgMjYwIDgwMCAzNTIgNDQ0IDYwMCAzNjggODAwIDI2MCA0MDAgNjAwIDM3MCAzNzAgMjYwIDUyMCA2NjYgMjYwIDI2MCAzMzMgMzUyIDQ0NCA4MTQgODE0IDgxNCA1MDAgNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNzc4IDUzNyA1MDAgNTAwIDUwMCA1MDAgMjYwIDI2MCAyNjAgMjYwIDU3NCA1OTIgNTU2IDU1NiA1NTYgNTU2IDU1NiA2MDAgNTU2IDU1NiA1NTYgNTU2IDU1NiA1MjAgNTM3IDUyMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA3NDAgNDgxIDQ4MSA0ODEgNDgxIDQ4MSAyNTggMjU4IDI1OCAyNTggNTAwIDUyMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDYwMCA1MDAgNTIwIDUyMCA1MjAgNTIwIDQ2MyA1MjAgNDYzIF0KPj4KZW5kb2JqCjI2IDAgb2JqCjw8Ci9Bc2NlbnQgOTcyCi9DYXBIZWlnaHQgNzE0Ci9DaGFyU2V0IChcMDU3aHlwaGVuXDA1N2VpZ2h0XDA1N0JcMDU3RVwwNTdOXDA1N1cpCi9EZXNjZW50IC0yMzAKL0ZsYWdzIDMyCi9Gb250QkJveCBbIC0xNjUgLTIzMCAxMDk5IDk3MiBdCi9Gb250RmFtaWx5IChIZWx2ZXRpY2FOZXVlTFRcMDQwU3RkXDA0MEJsa1wwNDBDbikKL0ZvbnRGaWxlMyAyNyAwIFIKL0ZvbnROYW1lIC9EV0dCTUgrSGVsdmV0aWNhTmV1ZUxUU3RkLUJsa0NuCi9Gb250U3RyZXRjaCAvQ29uZGVuc2VkCi9Gb250V2VpZ2h0IDkwMAovSXRhbGljQW5nbGUgMAovU3RlbVYgMTgwCi9UeXBlIC9Gb250RGVzY3JpcHRvcgovWEhlaWdodCA1NDEKPj4KZW5kb2JqCjI3IDAgb2JqCjw8Ci9GaWx0ZXIgL0ZsYXRlRGVjb2RlCi9TdWJ0eXBlIC9UeXBlMUMKL0xlbmd0aCA4NTEKPj4Kc3RyZWFtCkiJPFFdaBxVGJ1JMndTXce66xS6yuaikpdum1j8SVCD2cRmpbtbbQJFhaWzM7fJTXZn0zuzU7clDw0-JK6RbiLUmFo3DyFImwdfbAsuFUU0ELAK1VohhWJEUetDi99s7wjeUfA-3HvOd873cfiuLLW1SLIs48FDQ8lMaleKFFziUEPPkjJJjww75u5kYWLACjxx7yHZi7Z5D4c13suP371x97AC1-6HrQeuNrciUoss_3x7oDRZYXR0zMGP9_Z2J_De7u69uN8s5QkertgOKdr4RcsosckS0x1i7sH9hQI-GDTY-CCxCXOD4v8pMLWxjh2mm6SoswlcOiI0apJCnrBRwvAgKxsTRd02xqhFLNw_lMDkdaNQtqlLChVcoAaxbGJiZ4yVyqNjOE2tklOZJALkmc4qeKiYTyWwbpm4qFewSMnIKBU5mWiiFjYIc3TxjpcZtU1qOLRk2Xu69g2PBEOewCY5IomjSjulR6VOqUvqkUJin1Jaykt_yAfkivyhfKva8H5vyOJ-rNFabfNmmi_5Mwje5l9ovA8WFPgFccw_1SAg_nzIR7l_MfRxwZF3WQsQD5iKgUJCXoWN1lWgGmxAgm8g9eIUtAOBFuZCEdojZyHL-yH1G6TEk41egnenNEj89AkMXB-8tR7HXOh6MebHyx_tR4DZf813sN79vdxJbOc3TwWnw1FLz130kg9GeMDW6_Crnj0u9e897Ury0srK8sn9r1QPnH0aHnpm3j0hzdE5M16fW2t7vY87bqUuvUbcfVWFWKQhAh0QizSgOxtkeV5GIneaeyANQRdX96E7VfJj7vfCVJ415fOf_197K_Mt3xnnAs5y2PaRfRRXXybS0lHDvFIW_TOZbR2rjDy8kSRdhxCahUS8IgYr4pFXIBs64Ud3qBYw5uiyKCTMyGchIT_DFJhW91t5o4JVxK6ISmckPQycwi2-xkFDjdzPOmnZxEPe2mFm8L-X-avNsK0zDpDaLhr1phacQqH_nRAda9xxlDq37TkBAbeYC8RV_WplFycD5LFKnFpu5RX5gEToWEazM35z3P1sIHT_TzJ3h46fbReXPef_X09sgdg-k7oWW2qlTC7VaLRwGvFoLwH1vhVVv5kGvpP0jwAC5R51PCmVuZHN0cmVhbQplbmRvYmoKMjggMCBvYmoKPDwKL0Jhc2VGb250IC9NQ1NESVArSVRDRnJhbmtsaW5Hb3RoaWNTdGQtRGVtaQovRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZwovRmlyc3RDaGFyIDAKL0ZvbnREZXNjcmlwdG9yIDI5IDAgUgovTGFzdENoYXIgMjU1Ci9TdWJ0eXBlIC9UeXBlMQovVHlwZSAvRm9udAovV2lkdGhzIFsgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAzMDAgMzIwIDQ2MCA2MDAgNjAwIDcwMCA3MjAgMzAwIDM4MCAzODAgNjAwIDYwMCAzMDAgMjQwIDMwMCA2MDAgNjAwIDYwMCA2MDAgNjAwIDYwMCA2MDAgNjAwIDYwMCA2MDAgNjAwIDMwMCAzMDAgNjAwIDYwMCA2MDAgNTQwIDgwMCA2NDAgNjYwIDY2MCA2NjAgNTgwIDU0MCA2NjAgNjYwIDMwMCA0MDAgNjQwIDUwMCA4ODAgNjYwIDY2MCA2MjAgNjYwIDY2MCA2MDAgNTQwIDY2MCA2MDAgOTAwIDY0MCA2MDAgNjYwIDM4MCA2MDAgMzgwIDYwMCA1MDAgMzgwIDU0MCA1NDAgNTQwIDU0MCA1NDAgMzAwIDU2MCA1NDAgMjYwIDI2MCA1NjAgMjYwIDgyMCA1NDAgNTQwIDU0MCA1NDAgMzQwIDUwMCAzODAgNTQwIDQ4MCA3NDAgNTQwIDQ4MCA0MjAgMzgwIDMwMCAzODAgNjAwIDYwMCA2MDAgNjAwIDMwMCA2MDAgNDgwIDEwMDAgNjAwIDYwMCA1MDAgMTA0MCA2MDAgMjYwIDk2MCA2MDAgNjYwIDYwMCA2MDAgMzAwIDMwMCA0ODAgNDgwIDYwMCA1MDAgMTAwMCA1MDAgMTAwMCA1MDAgMjYwIDg2MCA2MDAgNDIwIDYwMCAzMDAgMzIwIDYwMCA2MDAgNjAwIDYwMCAzMDAgNjAwIDQyMCA3NDAgMzYwIDQwMCA2MDAgMjQwIDc0MCA0NjAgNDAwIDYwMCAzNjAgMzYwIDQwMCA1NDAgNTQwIDMwMCAzNDAgMzYwIDM2MCA0MDAgOTAwIDkwMCA5MDAgNTQwIDY0MCA2NDAgNjQwIDY0MCA2NDAgNjQwIDkwMCA2NjAgNTgwIDU4MCA1ODAgNTgwIDMwMCAzMDAgMzAwIDMwMCA2NjAgNjYwIDY2MCA2NjAgNjYwIDY2MCA2NjAgNjAwIDY2MCA2NjAgNjYwIDY2MCA2NjAgNjAwIDYyMCA2NjAgNTQwIDU0MCA1NDAgNTQwIDU0MCA1NDAgODIwIDU0MCA1NDAgNTQwIDU0MCA1NDAgMjYwIDI2MCAyNjAgMjYwIDU0MCA1NDAgNTQwIDU0MCA1NDAgNTQwIDU0MCA2MDAgNTQwIDU0MCA1NDAgNTQwIDU0MCA0ODAgNTQwIDQ4MCBdCj4-CmVuZG9iagoyOSAwIG9iago8PAovQXNjZW50IDkzNAovQ2FwSGVpZ2h0IDY4NgovQ2hhclNldCAoXDA1N3NwYWNlXDA1N3BhcmVubGVmdFwwNTdwYXJlbnJpZ2h0XDA1N0JcMDU3Q1wwNTdGXDA1N0lcMDU3T1wwNTdSXDA1N1NcMDU3VFwwNTdVXDA1N1dcMDU3YVwwNTdjXDA1N2RcMDU3ZVwwNTdmXDA1N2dcMDU3aFwwNTdpXDA1N2xcMDU3blwwNTdvXDA1N3BcMDU3clwwNTdzXDA1N3RcMDU3dVwwNTd2XDA1N3dcMDU3eCkKL0Rlc2NlbnQgLTI1MAovRmxhZ3MgMzIKL0ZvbnRCQm94IFsgLTE5OSAtMjUwIDEwMTQgOTM0IF0KL0ZvbnRGYW1pbHkgKElUQ1wwNDBGcmFua2xpblwwNDBHb3RoaWNcMDQwU3RkXDA0MEJvb2spCi9Gb250RmlsZTMgMzAgMCBSCi9Gb250TmFtZSAvTUNTRElQK0lUQ0ZyYW5rbGluR290aGljU3RkLURlbWkKL0ZvbnRTdHJldGNoIC9Ob3JtYWwKL0ZvbnRXZWlnaHQgNjAwCi9JdGFsaWNBbmdsZSAwCi9TdGVtViAxNDgKL1R5cGUgL0ZvbnREZXNjcmlwdG9yCi9YSGVpZ2h0IDUwOAo-PgplbmRvYmoKMzAgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL1N1YnR5cGUgL1R5cGUxQwovTGVuZ3RoIDI4MDgKPj4Kc3RyZWFtCkiJPFQLUFTnFb7XZe8CgTWwc7G5l9x71eoqAiJgFIcoAao1PngakJdZYEUElmVZF3nqJj54roCVh2hEngKRGBUiyBZEEVop2mHE0jCmNaFFQ9LWMT0X_3Wm_2qTuTP_3PP8v3POd36SsFlEkCS5dFdQRPD20DXbI4O26lSatPRUzbZM_YHUpAh9skewOiPV6sSLLCm62ojvOtAoFplelr7cJIXGxTDmdMf1rQFnQkKS7V03gjK1ubrUlAN6YZ3fxvfc8enn_fpcL7gL3l5e3h8kZyaqhYjcbL06I1vYrknK1GkzdSq9OtlT-CA9XQi3BmcL4epstc5gVWJQws-ohDewhNRsQSXo1CmpOItOnSzodapkdYZKlyZk7sc5sVKj0qdmalTpQmSuVr1flaQWgv5_EVZ7WmtauzXCahN8hWT1foIg8UfYkcRyklAShBtBeBGEj4TYSBCBBBFMEbsIImQREUERS3HLsBRCFBO3yAiyifwj-WRR2KJrEk9JiKRJMm-jtxmULpfmSkeoUZlC5icLkTXIntrG2Z62fWn3od15u9v2MvvAuTKzOG8m8flrs6TMRixeCLUUU2BCIzTaDKelMEchAQ3RYBUs1TILlfD6HzYjLFPiIG39Q1ZJLsi1yM2w8EMO-SVwEti5EEi7I3k08kRrGeRxC7nNosUckFQAOCWBO6xkwP0KuM-AC_dLIKzHkb04cAa5XEHuyJ1BK5OQewBysgbOwuJb4AYeDKyNBk93kHNyqCoDBfiBHOvlzuPgC9ngh9zgfcWLcZzmVSCMyu5OlF0x8ze7vxgaZb4LG3_vHFchU4iT5wcHp5lxjTmym-uIiyjbwyIlktPzVFuPqaiJv1hUm61j9Ebd4UIuN_tgRSaLSArJbcCeUrxo7SrDLk1FDdosJrMoU1_A5en2V2hYJKEwIiMsEcOBJjthm0R8JQbR4Jfr8y8UwKLEFWgDCkchP6JN8DF4AzEH687xyOuMdPsnB3ZiB8ej4Pj0cQ2sAum3rcFbynl5owEagQYveIccA1-J2CeaaXjHUoAr88WWeqBRPXjJ5EKjYSEwh8QdkMAhXDnytXr8DCcUlpCfgz-YwF8CXxvpuZkvp4f4r8Ybn00zj3XzMRPcVGRQhxeL9nnhWX64_qK3OZjfcmsqbY4VJ23MlwzxF_gL8ZWhe5mEE-H5Ki46N0MbySJpEbz7996quw3XefNnHe3DLL4OA24GewgCuRUMGGATBgTNNNhRL1CzFLZSSAG3pQ_bbkxOMxPpExFXuMsf7SrfzaKe14jFMZtvqI6-U3ltfFNhg0bLZBSmZeVzeZoU00HWBxe0xwgKMQacyKuwDlSwQSL2iF00vKVa3YMc2J2hhaq9fEyiLmIHg3ngkAJyHdfzyfTRcCZ8d158cFTLZA6HVqVKEdMJdhFAsw-m6nqH-aHrXfcmmPupdzCe2K6A-oha2-TqkFozMzRY0zt2RR92gcOzEPvBlYQQ6yAacJMtJuQKgZZ-8MWGgDfNpsV7OeQgBMJRTGVx3joKzuJDpZhiD6Vz2pw4Yxqb_PGZLwx8fnt5Xxcj-qBAi49sg8n_figXO_z46CQ7NXNueJi_c7t0eoqRi8Y3gzW9tJO8dLBm-xWltAQfz5CWUEgmJkrRKgpJLIlYXGaVNlLwm1eBUvCigBBjpBUU2FhipOBNyY1zuwxw8uHzH5w7MJ18wE1RIFa4fNpS0tLKgFE2Vnk7qoe7nLitNooVkEMKWlzAA18kNV9v7r_Yciyvjjt7uCTrEHNEtuv4ppNhh2x_q4uLDmAUA2v-HDv76KtLoz2coqA0q0JblWVbCNm0YiClPMOQxmXlafMz2MTstmsPn9fdruLlRnCEm2BDtmM--oMgEcvhER6f0eMRsmeRrweSIi_k9g9Egie8_WwKnGt5JJyWxp3ISoxiN5R--4fxM1_PDDUnxONUP635Hiafg38OrmktXiilYgBXVeSyEIiXuUDWn6Y-F86iVWq0GjnyigFkfw0JsHy8p_tWO1dBKQr8ZcfgAIaqK9Ue0XP6I7pcLRv46dzwmAkWfTNxakMMvuMFME9--p5swelDwVUiVkMuXf7g6KRqxHZEFdgVzCJvb-SI3NCaH5fiZy3gLri18Mj1jDQu4WDCjtSBUa68RVr6hB6pbLk5zt47Hx1p4q3rKjoB9TewO5bj_ADj_guki070LFDraijFXG9ZZ39TTVnxRQ4bpDLFXFV6YvkRdk-YIT6d16krPtrHhCLqrsUJlJh1Hmjb7D_B8a8z4DGT93RzjnMlLEEELEMu-FV8H5SrwQstB0Hx76FrgOjd3aphA6f4T0C0IVqzk0lo1V49zCmevx1vVGVEMUgG24tnRidPNQ1zI80DffcZkCg_867kdpikitkATfmeiqxq28NVxtpG5lxNbf1p7mRrZ3E7Ow90Nzj0HmnJ7OQV3zV2nmqra2fr6opL6njFXOMzmamwqLyQ3aI5tNnIYyBn9yVUx7OrEIN5xkP-NN3Xkx0dlZQVG5PWZr55ub2fx4jAzZJFR60wpaTFl_8ujTNU5dc0Mxfq689XcxMn5su6i20bSz4_rmWsi7kC7DAN8OIpYRkoJWIBJtWA6ffp17irGXsb4tjIME30Yb4EypESPzNKaKM6yrrq2riW2s66NvbeiY37eNQGSpkcKXG2ypw_wTJyGCeCBLGSRkpLJe70wk4bTKtl1p5bvaxraXUZdgGllXDUaxz_FcDO2YxvWImnesOMbdBKdVVcOnuJa6ptPtvKPjy2JYpHrVg_Iesru5f5PxarP6SpKAo7Xu8-XWukz0n64D2IZGpUqFlhoaR_FBoSWI2aGmFSmr8Wsk0xdYSpqDkTUSeE4c-p1SyzZpSJzAol_GNJDKksCh1Wf1icJ6eg-6w_D_ece-5vnO_746J40VZDqNw9EJhWpHUQPh3yyhxWts3WF5TDdCpS4I4Bnpkhw73JSCHUaj_iGr6hGP8wEEkxCSAGuOlphjd3d6qgg6pK6_ZmBuRUac_dLAJtns89hWfqKX-Cgnr4F3dsx7ioUfHw15-iarnmM7ZOOygMHT2dg8Jb1t2Ggz1yCSbTN13rkkVgzcnRiL4NaQt6P4cpmWWZi6nx21-JkCUG0QIlvil17WvrkyLnnyj85iQllucWaq0sLbIaY9v4ui0hMxSIFz8wiwFUoERznBfur7CllqRd1HArzan24syK0-H07IFP6ddsf2AT6sqJ4RTDwlj5ExZp_NfThvTCxiAqhwMTn9oeGmWRmo-VN6rCxyod9bmR-Rl2EqyTnVP2UTky1kkA98v_RJmZtpH3NLEcN_kCyhHDwb4I1zuNufTMYuxk_44bKBOMMsCJeszvV8prMmtOqo58QRDbVUYzcZh0HssI7zrbGPGeL5YOjpT-0gYuN9ya0gChmuosDZUCSduON5IkAMqShclIYBK7i5LyDTtUVTI561yRZhspzPTz_FPTM3FtjKxqKrEWiAY6h6MS-ClK17uXL1x8LyYPfS45rmwMDU61ytRZbLu57TeTQBVbfJFRi5UppAhUbgYDYtUIAm1HTtLPVULy__S7BuFzIZJSQskB3Ce-kEKzLMokCSco0EizLEYSyBWKQshPvBRb_Chj4UdytzCpBmqLaoO2c7I15VNeJKIYiKILCYTiMRqPIL9LISSdUj9iaksJBNMhn5Wa2vfSGnHk-0E-lp_Z7dx-POQCUu6AsCogb9ViC3NRrQd2q2NWm0sitU3q37K8AAsbSreAplbmRzdHJlYW0KZW5kb2JqCjMxIDAgb2JqCjw8Ci9CYXNlRm9udCAvVEhHVENCK0hlbHZldGljYU5ldWVMVFN0ZC1CZAovRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZwovRmlyc3RDaGFyIDAKL0ZvbnREZXNjcmlwdG9yIDMyIDAgUgovTGFzdENoYXIgMjU1Ci9TdWJ0eXBlIC9UeXBlMQovVHlwZSAvRm9udAovV2lkdGhzIFsgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAyNzggMjc4IDQ2MyA1NTYgNTU2IDEwMDAgNjg1IDI3OCAyOTYgMjk2IDQwNyA2MDAgMjc4IDQwNyAyNzggMzcxIDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiAyNzggMjc4IDYwMCA2MDAgNjAwIDU1NiA4MDAgNjg1IDcwNCA3NDEgNzQxIDY0OCA1OTMgNzU5IDc0MSAyOTUgNTU2IDcyMiA1OTMgOTA3IDc0MSA3NzggNjY3IDc3OCA3MjIgNjQ5IDYxMSA3NDEgNjMwIDk0NCA2NjcgNjY3IDY0OCAzMzMgMzcxIDMzMyA2MDAgNTAwIDI1OSA1NzQgNjExIDU3NCA2MTEgNTc0IDMzMyA2MTEgNTkzIDI1OCAyNzggNTc0IDI1OCA5MDYgNTkzIDYxMSA2MTEgNjExIDM4OSA1MzcgMzUyIDU5MyA1MjAgODE0IDUzNyA1MTkgNTE5IDMzMyAyMjMgMzMzIDYwMCA1MDEgNTU2IDUwMSAyNzggNTU2IDQ2MyAxMDAwIDU1NiA1NTYgMjU5IDExNDcgNjQ5IDI0MSAxMDkzIDUwMSA2NDggNTAxIDUwMSAyNzggMjc4IDQ2MyA0NjMgNTAxIDUwMCAxMDAwIDI1OSAxMDAwIDUzNyAyNDEgOTI2IDUwMSA1MTkgNjY3IDI3OCAyNzggNTU2IDU1NiA1NTYgNTU2IDIyMyA1NTYgMjU5IDgwMCAzNDQgNDQ0IDYwMCA0MDcgODAwIDI1OSA0MDAgNjAwIDM5MiAzOTIgMjU5IDU5MyA2MjAgMjc4IDI1OSAzOTIgMzY3IDQ0NCA4OTIgODg5IDg4OSA1NTYgNjg1IDY4NSA2ODUgNjg1IDY4NSA2ODUgOTgxIDc0MSA2NDggNjQ4IDY0OCA2NDggMjk1IDI5NSAyOTUgMjk1IDc0MSA3NDEgNzc4IDc3OCA3NzggNzc4IDc3OCA2MDAgNzc4IDc0MSA3NDEgNzQxIDc0MSA2NjcgNjY3IDYxMSA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA5MDcgNTc0IDU3NCA1NzQgNTc0IDU3NCAyNTggMjU4IDI1OCAyNTggNjExIDU5MyA2MTEgNjExIDYxMSA2MTEgNjExIDYwMCA2MTEgNTkzIDU5MyA1OTMgNTkzIDUxOSA2MTEgNTE5IF0KPj4KZW5kb2JqCjMyIDAgb2JqCjw8Ci9Bc2NlbnQgOTc1Ci9DYXBIZWlnaHQgNzE0Ci9DaGFyU2V0IChcMDU3c3BhY2VcMDU3Y29tbWFcMDU3aHlwaGVuXDA1N3BlcmlvZFwwNTd6ZXJvXDA1N29uZVwwNTd0d29cMDU3dGhyZWVcMDU3Zm91clwwNTdmaXZlXDA1N3NpeFwwNTdzZXZlblwwNTdlaWdodFwwNTduaW5lXDA1N2NvbG9uXDA1N0FcMDU3QlwwNTdDXDA1N0RcMDU3RVwwNTdGXDA1N0dcMDU3SFwwNTdJXDA1N05cMDU3T1wwNTdQXDA1N1JcMDU3U1wwNTdUXDA1N1dcMDU3YVwwNTdiXDA1N2NcMDU3ZFwwNTdlXDA1N2ZcMDU3Z1wwNTdoXDA1N2lcMDU3a1wwNTdsXDA1N21cMDU3blwwNTdvXDA1N3BcMDU3clwwNTdzXDA1N3RcMDU3dVwwNTd2XDA1N3dcMDU3eFwwNTd5XDA1N2J1bGxldCkKL0Rlc2NlbnQgLTIxOAovRmxhZ3MgMzIKL0ZvbnRCQm94IFsgLTE2NiAtMjE4IDEwNzggOTc1IF0KL0ZvbnRGYW1pbHkgKEhlbHZldGljYU5ldWVMVFwwNDBTdGQpCi9Gb250RmlsZTMgMzMgMCBSCi9Gb250TmFtZSAvVEhHVENCK0hlbHZldGljYU5ldWVMVFN0ZC1CZAovRm9udFN0cmV0Y2ggL05vcm1hbAovRm9udFdlaWdodCA3MDAKL0l0YWxpY0FuZ2xlIDAKL1N0ZW1WIDE0NAovVHlwZSAvRm9udERlc2NyaXB0b3IKL1hIZWlnaHQgNTE3Cj4-CmVuZG9iagozMyAwIG9iago8PAovRmlsdGVyIC9GbGF0ZURlY29kZQovU3VidHlwZSAvVHlwZTFDCi9MZW5ndGggMzcyMgo-PgpzdHJlYW0KSIlkVQtQU2cWTiD5E16XhXgRE8i94gN2BSSi1NQniCAsDxV8P0NylSgvk4DgYzs6jhVprFPdrq3PtqvuVutz0WpBwUcNihXFanXbtVt06u6oHR3lXHoys3uuUmenO8n853-c__zfd15XrdIEqNRqdWzxpOziCRlDJkllNZLHabcVSNVSXnGRx5GU4VAUBNmklvto5JhQHt_Eip8v_2zXwg_hsiaiLeY3DyNVAWr1s_9MqKyqczkXl3pEi3XkyEQarSkvx9REcVhKyjAx3VFZIolFdW6PVO4Wcyrsla6qSpfNIzmSxfSyMnGqctktTpXckqtG2XyNR3S6RZvocdkcUrnNtVSsXERnTodUViK5FksuMdNVbV9abnPbS50VUoWYnp0oSrX2smq3s0YqqxPLnHapwi05RE-pq7J6camY56yo9NRVSTQpcdlcdWJ2ecmkRNFW4RDLbXUioXRJi52E00WXnBWiXXJ5bCSXVLucbofT7nFWVriTh2YVFStGhosOaZFKpaafKixAFRGsitOrhgaoUgNUI1WqDL0qT6sq0qk8KtVIcreqQDWF5g2q3arHaoNaUncFGANqA-4HxgQe1PTVjNK4Nd9qh2qrtNu1rSyBTWNL2YfsmS5N9ye9Xp-gd-n_FTQuaH3QveChwYuCvw3pFzI2xBOyMeRR6JDQqtBrYZFhnrDDYU-5WM7G7eC6wjU4sKFZftSspnFgc2CDRt7QM9m_gcEmvMjjWNiihYcMRWzhQVn439P52fyXcxiLtGbyWV6ZobLi_AH0b4BktQ8sgb6oTuZdULbQtk4_QffPbdd9PxohGS06DtdAKXDqFmgNbIFSHlqBw1bWe_MS3bwUBRZFlXGrIUheC0HqPRAHKyAucA-84PEABKeCGUQI7oIDsBeDuwigGYNTca8Z4qJuNsA4YKBvwHEZo2lAPTLaumvm9tTIs7tBWK6GTWRLjumZzKOWrRJTnAlevQiZ_tkQp-MagJOXEL4jEANlpAfp8gT-630_3POdtOWYN454Y2G-DcOsCVkbMU0vv0_g77Eznd8d72jSdzSdv_jcCLGo-Q774OgUDMfhm80KC1jyU20NVEFQ5E4wYTCYwE6SgwTDacMdqIXrfMdDb_-cWQt-b5Xabm8Q0MRwxIYXSWAxQSyE_wPiBcOpO3OvjN1h9uoMJ9p3HvzyqhEiMLwDKYTJCThgvRlMrHPj0ZYbprOnnZMFw91RqW8VbFZYQxq50Uq028EUCO1R9HraQwhKlq1oYHjBn6jFKGjSQZAG8xjkwNvaTS9dD7u6u8n5sqh4i_KC95tYlr-vFvqwo_v37zxjuv35lIyJc3PT8-edaFsnYBzDeC9w4yHGBP03QAgEwkAj9B3TjebiBasckrneux9StdD4yvq2bojtptAKsFp5YLe8g4fwlZYbGGbCOeNwBoVu4gXMp2QZ_AS0MODciouLTggn7QX7C03ZMz3FDqHeq73z7j4wPTM1HbbP3SoMQgt_87OcnDxHbnamrb2944jvtqCEvR2iI4_JLn8EJBq-lz9QqLgwGr5nhg7sh8NH4LyVaNdDIvNC4d9hO8RDHz23uhtW_LuuBmq7Iz-AONQCT1AoYyCOQpZCmdt57ljH1db5kybOmJOdXXS602xopNMrfNahomu15nqd4ZR1-ay8YUYcAn2zyelREH0T4rvmXBm5S4ni6asfHT7fYbxWCKGYodxlyOVipGC4Bfwf-LbmQxe6dhQsLpy7dMrUWZ_6zAqe2G7Y1lsOq0EIPAiTeAzf_iATwkww5zbMoMyfOBXysRQHJ6AWBxTvmHJkobDguK_ykunamY_PHRO89doxb1ehaaBp9uLjp9cKT8HCZzivXr18rP3ajcbc3KxFBeOEV7UIc19Xcm89kujm0YLJQLX5N0qsVEqs3OXqdT2FgeuiIJWdV8K7m7UiiTcY7u6Zr8UR7A5m8l72AD_UYj7jLlGBwcxuGAVcZAdZ3AKJGA4Ww3OogRs8jGNfnarISJ-2PF_A0Ywe43h4wsj34W0w6P6Cr4b_WTDIbX891HTdCP2GnsQYM94nYIpaPDt_dnV2YYmrWMBUxj1aAdFyLUSrjxKHnny5ggdrLUb_iBYTLsMotJKbSoAkLIMJEE49I2u7gIks6a1pOUNM2H86hFIOR7SQ6H_n2rTMPwrcowYyOZ1MKsBvkNmOKPAxqHhB3rLDGhRAj4TbpwCK5kHDIKV5KA7EYcOLUgQUGfcuGaDHOEh6ZYQMyMcg2s_T5hNqJU8gSQ6j9aeMm0n-_YI0ra816c0v4CxpnaXNx6S1kXGgI5ZJNfARFZSTspy4yirK0CbDKvlx1E9yFLZgEAxjhqaEtZOLB_2K1tdflozeKcDvGKStQPMDHPx_nuno3HfjHBEnMD6Cvm_5S-qNv-DxQSNSivTMp0Dtw0bYp3RtfLqnpmf-8teolVOmmHi1GQ9tEK8cxMsuL4Nmv0v7Uife765neEZ2azGFiBn_J3xyNJk6qiTfLwFbRtVkhRIopdZlJdC9sM1gierlBv1bMBQjMGQ6if69vM3cx8RkK7lVafEKlW2Q-IrKVrBSBnRQp9GDPgu0mCXgrd7M4tjN9pUpKbkrMwTkGHeHbBwADczrtXHwF3ccoMK4-5cl498Xtkytz9o1X0_ryuPv3Nvw-frO6ltjd-u9ursHPrt8zwiqN-9jH7Of731gPLt-tDxrglQ-UcDxjNu8gj5C71CBfELF0QcSYS81rm8MDz-BNTxm1kPCGBhmgunPHkNfSMPhB2dsEwzfYPYp53vnjJcu7mm-5asesdlM_Yx8E9uOv0UezRNxMBq7S15Ump9X_pfvag-KqozirMu9y6uNvK7RXr1XF0oIEg3Y5bExqYEJhQO-TcnHqIkPyh6rvCQa0UUeghig2EjEsOCTERQfGeigUVFqUxggbA-EVsFU3HPpbDOdCzRN__THndmz93znnufvdz57RqR2tiF1ocD1G2adsonqJALJDso1mCgUMFEsJCZRi90mmEyiajSNVi__z0jln7OGEzU4nUVXRyZDFVNLmQxOY9HTkSKLjJTCyFgR_1ciQ5AA7iRTmZ-gt0B1jUq3hXcB2MYfJRBLpE5tlDxghWbz7iBUrlxT2vTF6YNni4Wz-z_PrTzignFDmiv7TlQ08583ZMXFh-cHZ4hBO4OS0VU79aflA1-1lZ9vFLjtuSkFKftSXcCHzTiSff68FipU4Imen_oQ97uvxWc-EtRJ6fbp_dBjh1jT-Gb68oejM9JMZEiz0ZanSqj96r12_iEouyEB4lHZ7RMb--7it0QzlBKxSU86VmhC1hsNhvUdP_9yqqOn55QxWKBg7GSTOpT4qwF-1YBXFjp_jWoejf7ojiEYOITjiCsm2X6EyYdFnMy-lrVu9Tz-xa2__dBe03fn6qnN8cXif5yT05I9NsAR5B0Nhg5LzeylTfOr4ngMiKRaiiJ3Eb3bUQsz2loqrxwT8lgzLNbok8NeCFpv7bIe7ezvq4kwjPr3dC-Msysssov18J4Ggg7owRdf4XHmFHTDQAz-A53BAK5WQpbTIj7Fbspa-VYCb3yzvne32Ot8Oq_qRi9_vXbdnFJRjV9Tn4Tbeu0KK9mzTgSdFD5g86tlzxw6VHe4PMdcLjxW5b77Tv42Hl0S5s0Q1-oDrCp1Ei61gYsdLvdPtxNC6SATeK6LuyddkMo0tOpwXdFm1bWNSz6dy-OzkahGLeraaEsKbG2taq4V8-ays3I3LDLyy1YdbsoQcSqLbkXg_sYDHrJqwMcqYvFvmuDkrh9unbjR2Wd52TBrbXS4KAOoZKApjTUp5IqvJI-p3DqoZIeOnbnV9tlqvYCNJN9UwYR1VvRf_Hpy_AZhD1RQvdWOiXS6zHSf2LaFDrbIoZbRC9JfpqGKuAIpRY3owBaT4lvS-VY2Hs1ebqJlIZWhkcYt0EmcnMDIRDYX4xiJQ1owZdPyLP1jV64wTdhB2d0BP3AeTy8gZvSZMIm7KOtAE_vHd4ere3dHzxfwDMn1qm8OtnV2FSZEC3iO5EEVaI2XcNycRWsStwityfGVMfyrizYvXi-aKb03UKsazcfIB8bSwTX-bz7mj-aD5W59j7xKbtKxXocC8rwOPtZg6BA60XZpGAInCIUwHTihAfU6dMIweRXvtnTfvWcJDQ3bGvq8_9bu2zSGGPIBGMmQ3AnNY63e1YwxGvqdk8cuPX717VYe_NuJ8AWYNhueRn3sgm0LN8ph3GsjP6DMYdQEbursEbmunpru_jvH9GECd8-wJWK6oMb9FKM3uFHveGWPDDpXOhJjOstZ7pz88VH5EfOeUoErBS8VZ8nP2pn3IR8dlRjzjmz-y7tymFPSbYROino6FEthWsCowcmF4JoIHjy4gdtZmHQ3vCb0EzH8k4iiS9qG84VHLzWkLSyQ12_gMr3rkOFRiexG9Jpxe-vtHeKD96-mrdIuW5K6ZsGSz1pzyMnGXdSV122KPvl2kTEcLPdTYEjAz9jC-rSbfqmuzS2oFECp2pOWnpPGv7Gz5IIIxY9s3lTCdHD2G5AMpn9LKJ2UZ0iHlawuaVXkvG0NVgEaSb6pwgl1evC_1lrdclLIZ7ntc4DCQ8-RnlXsk4qUUol8lKZJgY9l5FbAYwb9WIhylDG0MqghclS3cPiMcviirPscG4APSTUIHjIosCH4iIQAeMQgrRCh8rFp7K9wn_C-D-8z8ByrnkJUee4DMJsUe6XjdL-UGcGJ4IYahQE9iwugmm6T2TJjzER-BvAM6lm6UZoZNXF3DO2XHSbFLqlaKVXIDhDNMGiXnXUFO0PLGsQ4dhxak7p0efYzyTnJBW_nuaCrao_FYrbwAxePd9cSZnlDNAwrvodhJT0a346IwcHOzsHBiA5fX6PRV1CnlA8nlmNSGby0l4Xy4gfFjltlqrE_S1ygquh-keP3Ele7G-jc7Qc8PEB30OOJXA-11DRh2EvztwADACyda7sKZW5kc3RyZWFtCmVuZG9iagozNCAwIG9iago8PAovQmFzZUZvbnQgL0FLVEVXTitVbml2ZXJzYWxTdGQtTmV3c3dpdGhDb21tUGkKL0ZpcnN0Q2hhciAwCi9Gb250RGVzY3JpcHRvciAzNSAwIFIKL0xhc3RDaGFyIDI1NQovU3VidHlwZSAvVHlwZTEKL1R5cGUgL0ZvbnQKL1dpZHRocyBbIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgMjgwIDEwMDAgMTAwMCAxMDAwIDEwMDAgMTAwMCAzMzMgMTAwMCAzMzMgMTAwMCAzMzMgODMzIDU1NiAxMDAwIDg4OSA4MzMgMTAwMCAxMDAwIDEwMDAgMTAwMCAxMDAwIDEwMDAgMTAwMCAzMzMgMzMzIDMzMyAxMDAwIDEwMDAgNTU2IDEwMDAgODg5IDgzMyAxMDAwIDc3OCA4MzMgNTU2IDEwMDAgODMzIDEwMDAgNTAwIDEwMDAgMzMzIDEwMDAgMTAwMCAxMDAwIDg4OSA1NTYgNTAwIDUwMCA4MzMgNTU2IDUwMCA1NTYgMTAwMCA4ODkgODMzIDEwMDAgODMzIDEwMDAgMzMzIDMzMyA4MzMgMTAwMCAxMDAwIDEwMDAgNzc4IDI3OCA1NTYgMTAwMCA4MzMgMTAwMCA1MDAgMTAwMCAzMzMgMTAwMCAxMDAwIDEwMDAgODg5IDU1NiAzMzMgMzMzIDgzMyA1NTYgNTAwIDU1NiA2NjcgODg5IDgzMyAyNzggODMzIDIyMiA1MDAgMzMzIDUwMCAxMDAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAxMDAwIDUwMCAxMDAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgMTAwMCAxMDAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAxMDAwIDUwMCAxMDAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1NTYgODg5IDEwMDAgNTAwIDUwMCA1MDAgNTAwIDUwMCBdCj4-CmVuZG9iagozNSAwIG9iago8PAovQXNjZW50IDc2NAovQ2FwSGVpZ2h0IDcwOAovQ2hhclNldCAoXDA1N3NwYWNlKQovRGVzY2VudCAtMjI3Ci9GbGFncyA0Ci9Gb250QkJveCBbIC03IC0yMjcgOTg5IDc2NCBdCi9Gb250RmFtaWx5IChVbml2ZXJzYWxcMDQwU3RkXDA0ME5ld3NcMDQwd1wwNDBDb21tXDA0MFBpKQovRm9udEZpbGUzIDM2IDAgUgovRm9udE5hbWUgL0FLVEVXTitVbml2ZXJzYWxTdGQtTmV3c3dpdGhDb21tUGkKL0ZvbnRTdHJldGNoIC9Ob3JtYWwKL0ZvbnRXZWlnaHQgNDAwCi9JdGFsaWNBbmdsZSAwCi9TdGVtViA3NQovVHlwZSAvRm9udERlc2NyaXB0b3IKL1hIZWlnaHQgMzMwCj4-CmVuZG9iagozNiAwIG9iago8PAovRmlsdGVyIC9GbGF0ZURlY29kZQovU3VidHlwZSAvVHlwZTFDCi9MZW5ndGggMzk0Cj4-CnN0cmVhbQpIiTxQz0sbQRTe1biCFqGFIKiHB3praq1VMBchqFVRgjWW3gqTnWd2zO5MfDMx7t1DZKGIPfV_8D-QoBf_kl566KmX3XQD7ayHXr4fj_c9Pp7rlMYc13UXawcnO5_rrz9JcYGkWdgw_E0de7onTLClouhIFGtz2bybzZayhRflq7z353x4M5H-ePk9q75yxlz3998t1YlJtAID76obGxWL1fUKrK6srMIzva9x1URoxNpgpGFf-oo6iphBvgy1MITjIqzhGDXSRTH8XwiEBgaGGMeIURvUKeyh4Bg2kVpIsE1dvx0x7QdCooTabgXw0g-72ubDGELho9TIwQSkuq0ADoVUJu6gFU1iFMNu1NyrAJMcIhaDbUnYErYn2ZCQ4CMZZvmsS0Jz4RuhpF5--6FxUhxZA46njlN8of4x6yeD7NfAtbg0GE9KWX94NOp76df8qZxvpt8m0p9eDvljOS3M6HZy5H151ulmbr2XPZQLlRduJr2bSZLhdeLlvWTyfup--p8AAwBzMqizCmVuZHN0cmVhbQplbmRvYmoKMzcgMCBvYmoKPDwKL0Jhc2VGb250IC9aUUlESVArSGVsdmV0aWNhTmV1ZUxUU3RkLUJkSXQKL0VuY29kaW5nIC9XaW5BbnNpRW5jb2RpbmcKL0ZpcnN0Q2hhciAwCi9Gb250RGVzY3JpcHRvciAzOCAwIFIKL0xhc3RDaGFyIDI1NQovU3VidHlwZSAvVHlwZTEKL1R5cGUgL0ZvbnQKL1dpZHRocyBbIDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDU3NCA1NzQgMjc4IDI5NiA0ODEgNTU2IDU1NiA5NjMgNjg1IDI3OCAyOTYgMjk2IDQwNyA2MDAgMjc4IDQwNyAyNzggMzg5IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiAyNzggMjc4IDYwMCA2MDAgNjAwIDU3NCA4MDAgNjg1IDcyMiA3NDEgNzQxIDY2NyA1OTMgNzU5IDc0MSAyOTYgNTU2IDcyMiA1NzQgOTA3IDc0MSA3NzggNjY3IDc3OCA3MjIgNjQ4IDYxMSA3NDEgNjMwIDk0NCA2NjcgNjQ4IDY0OCAzMzMgMzg5IDMzMyA2MDAgNTAwIDI1OSA1NzQgNjExIDU1NiA2MTEgNTc0IDM1MiA2MTEgNjExIDI1OSAyNTkgNTU2IDI1OSA5MDcgNjExIDU5MyA2MTEgNjExIDM4OSA1MTkgMzcwIDYxMSA1MTkgODE1IDUxOSA1MTkgNTAwIDMzMyAyMjIgMzMzIDYwMCA1MDAgNTU2IDUwMCAyNzggNTU2IDQ4MSAxMDAwIDU5MyA1OTMgMjU5IDExMzAgNjQ4IDI1OSAxMTExIDUwMCA2NDggNTAwIDUwMCAyNzggMjc4IDQ4MSA0ODEgNTAwIDUwMCAxMDAwIDI1OSA5OTAgNTE5IDI1OSA5MjYgNTAwIDUwMCA2NDggMjc4IDI5NiA1NTYgNTU2IDU1NiA1NTYgMjIyIDU5MyAyNTkgODAwIDM0NCA0NjMgNjAwIDQwNyA4MDAgMjU5IDQwMCA2MDAgMzkyIDM5MiAyNTkgNjExIDYwMCAyNzggMjU5IDM5MiAzNTYgNDYzIDg4OSA4ODkgODg5IDU3NCA2ODUgNjg1IDY4NSA2ODUgNjg1IDY4NSA5ODEgNzQxIDY2NyA2NjcgNjY3IDY2NyAyOTYgMjk2IDI5NiAyOTYgNzQxIDc0MSA3NzggNzc4IDc3OCA3NzggNzc4IDYwMCA3NzggNzQxIDc0MSA3NDEgNzQxIDY0OCA2NjcgNTkzIDU3NCA1NzQgNTc0IDU3NCA1NzQgNTc0IDkwNyA1NTYgNTc0IDU3NCA1NzQgNTc0IDI1OSAyNTkgMjU5IDI1OSA1OTMgNjExIDU5MyA1OTMgNTkzIDU5MyA1OTMgNjAwIDU5MyA2MTEgNjExIDYxMSA2MTEgNTE5IDYxMSA1MTkgXQo-PgplbmRvYmoKMzggMCBvYmoKPDwKL0FzY2VudCA5NzUKL0NhcEhlaWdodCA3MTQKL0NoYXJTZXQgKFwwNTdwZXJpb2RcMDU3c2xhc2hcMDU3ZWlnaHRcMDU3QlwwNTdFXDA1N0ZcMDU3TlwwNTdXXDA1N2dcMDU3aVwwNTdtXDA1N29cMDU3clwwNTdzXDA1N3ZcMDU3dykKL0Rlc2NlbnQgLTIxOAovRmxhZ3MgOTYKL0ZvbnRCQm94IFsgLTE2NiAtMjE4IDExMjkgOTc1IF0KL0ZvbnRGYW1pbHkgKEhlbHZldGljYU5ldWVMVFwwNDBTdGQpCi9Gb250RmlsZTMgMzkgMCBSCi9Gb250TmFtZSAvWlFJRElQK0hlbHZldGljYU5ldWVMVFN0ZC1CZEl0Ci9Gb250U3RyZXRjaCAvTm9ybWFsCi9Gb250V2VpZ2h0IDcwMAovSXRhbGljQW5nbGUgLTEyCi9TdGVtViAxNDQKL1R5cGUgL0ZvbnREZXNjcmlwdG9yCi9YSGVpZ2h0IDUxNwo-PgplbmRvYmoKMzkgMCBvYmoKPDwKL0ZpbHRlciAvRmxhdGVEZWNvZGUKL1N1YnR5cGUgL1R5cGUxQwovTGVuZ3RoIDE1NjQKPj4Kc3RyZWFtCkiJbJV_UFRVG8fvZfcc8EfX3MtlkLXd25Ig8UNAK-X1lUgU8UeC-INQt9bdK6zArt5ddlkD0WZKd5BeX14Z2zcMMi0VM8bICqKogYZ-qTlqjv1wppqJl7cmq3d47vqsY-da01_v_eOc85zn-3nOc855zlyeM8ZxPM9bq8pLi0vLMpcpdQHF73Y6HlUalJVrK_yu7EdcpX5dYtPMvJZobJkap82cKmEBem7euukk8MM0LWH60EzabeLieP7X24u920Oqu7rGL-ctmD8_i7ULcu-0c7Pk_NzcfLnI5d2iyBUhn1-p98mlHqdX3e5VHX7FlSMX1dXJa3TYJ69RfIoa0Cf_Skp2-2SH7FcdLqXeodbK3q3M53YpdVsUtVpR5WK1wVlb7_A5a9wexSMXlWTJSqOzrsHnDih1IbnO7VQ8PsUl-2tUb0N1jbzS7fH6Q9sVNtiiOtSQXFK_ZVmW7PC45HpHSGZZqkq1m-WpMsjtkZ2K6newfluD6va53E6_2-vx5cxZWrFWDzJPdilbOfZN4-7mZnCpXBqXzs3h5nPLuOXco1wZV8Gt5TZwldx0duRcCWfngnwC38if4b-PS4lzxF028IZmw7BRMO4yniZ0Q-ug9uMgfzlJ2xcti-2j8CyOSLgI_kVgjKKMQxLoRqw9Pkbtd8awCJlNtfckfYS6JeCNVsjhd0GFYVcS5NNTkEMwnwr_CER3B_mwdo8hvFMCSqEnupsgpcKS5gnYOR4KQOOEKQi2dSDCarClgSj2gwCbJfHaV6N9Vz78tLrwkXX2-Raxf0VZ3zmreHYdjEplZzYO-C3heLE_w79orZyCU4F_ADLhPpC_htTPq66sPmRpixff_vL5Dy6dS_ntwes4wyKexdmYsAgTreJlEJNG3-y9NDLkKS2vqC7fWHrkvEX4bytbt3LCH4BqEEzaHshDCfLEq1ALFyXIocPnnn64cIOr2IqPUVyCgqRNp5AKSddgjlV864IyUnJYX7Tvesfgla9SYGb-SUy14BAFE9NCAX23p3GdVby0qqp2jQWXUqGvFZJhMVszG5L5vdEVhr1J2gB9BZJJzEJ7YTEBpMdAIIi0B7KJlkxf1n1vUGFbdwAGGLjgDpjOQMijH8MAgZE_iA_paVhA4Fd6XCcOUAHiuwNRe5Cp9-rqdPqLFiZtFCpjYcJYyIzaCd5HZ8UOkDDF9doBgg8y6uIflNYB1HAzHLVLmEfREzvMRCHtMMH7KQZiXczao3UR5nvnlp2wuwdVa2fRD8faCeRSoQkrx4GfgI8mmoOmTrCJYzCsLZQunNq0fG4V8rNK64dGvxkE47iVHfaZMQlnUnHsyYUs7HlP-dFiMxZk4QOYieIXD8G0kf6X3um1toVJmnfHwjzz5qoX32uzoo3i3ZBg_x8UXvsM8k5bhJihO6BFgj_DJL4FBgwtSZBKIU6LEGT9KnhMYv0OnETYtACvMu1DP2WA0dQS1TADMsReBtjoCJwgH0fOXvvC_NoxX92ecEu4yVqCX-qQ7hKP_1-njX4Cw-SfJw-e6kn5vvCNgtKiretrLB-1VL6yyly0wlP-hDVMxQvPXyJsn8I29gquT_AdYIN2sBk6QJFw9tjDrJLLgLB2OWTM_hxtuBIJpuNiC9iSvn0bEn-7fMK-dN5mTLp3Wc0HoxaBbSAVJuNkSN4XNLVoa8Tn7mxgAEJEHP3uhTNfA2_u6npmb5dVfO44WFjqbU3N-5vMhUuU4lpruI2puscImqkgN48DjJueguS_sZxYEPGq1qjFSTCjLa0HiTkdJQcKBX3ei34r8DXk5J6u3dUpmzY1PWERx6rWH-m3dmJyIB7vginOGzAFxAFI_Hb1a4VHLeLV4iNk4793Ro6k9PZGBt8_uuPx_RZBboUMTNTvynRC6xC3a69rEYkVltiLK2JGVgO1mpFgGj0fixDQa7K8FbIxG3LRHIDbkNMaNGmZN8Ss6JU_uZXsh3GLcU_CLb2kxf04N8YxO6hxBGfRYT1OBhXnwTy4rRcp3iYwmwo7O6P2TtwWgb-3Uug8-MvB2NVI_J-ThxLgWPvP7bH_HJo0MRlsU_ZPFbQZiVEq_S7AABLoEHkKZW5kc3RyZWFtCmVuZG9iago0MCAwIG9iago8PAo-PgplbmRvYmoKNDEgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgMzYgNTQwLjAwOCAzNzQuNCA1NTQuMDA5IF0KL1N0cnVjdFBhcmVudCAxCi9TdWJ0eXBlIC9XaWRnZXQKL1QgKE5hbWUpCi9UeXBlIC9Bbm5vdAovViAoTUlDSEFFTFwwNDBBTEkpCi9BUCA8PAovTiA3NyAwIFIKPj4KPj4KZW5kb2JqCjQyIDAgb2JqCjw8Ci9EQSAoXDA1N0NvdXJcMDQwOFwwNDBUZlwwNDAwXDA0MDBcMDQwMFwwNTY1MDJcMDQwcmcpCi9GIDQKL0ZUIC9UeAovRmYgODM4ODYwOQovTUsgPDwKPj4KL1AgNCAwIFIKL1EgMAovUmVjdCBbIDM3Ni40IDU0MC4wMDggNTc2IDU1NC4wMDkgXQovU3RydWN0UGFyZW50IDIKL1N1YnR5cGUgL1dpZGdldAovVCAoQ291bnRyeU9mQ2l0aXplbnNoaXApCi9UeXBlIC9Bbm5vdAovViAoVW5pdGVkXDA0MEtpbmdkb21cMDQwXDA1MEVuZ2xhbmRcMDU0XDA0ME5vcnRoZXJuXDA0MElyZWxhbmRcMDU0XDA0MFNjb3RsYW5kXDA1NFwwNDBhbmRcMDQwV2FsZXNcMDUxKQovQVAgPDwKL04gNzggMCBSCj4-Cj4-CmVuZG9iago0MyAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL0ZmIDgzODg2MDkKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDAKL1JlY3QgWyAzNiA1MTYuMDA3IDU3NiA1MzAuMDA4IF0KL1N0cnVjdFBhcmVudCAzCi9TdWJ0eXBlIC9XaWRnZXQKL1QgKEFkZHJlc3MpCi9UeXBlIC9Bbm5vdAovViAoOTNcMDQwTXVsYmVycnlcMDQwQ3Jlc2NlbnQpCi9BUCA8PAovTiA3OSAwIFIKPj4KPj4KZW5kb2JqCjQ0IDAgb2JqCjw8Ci9EQSAoXDA1N0NvdXJcMDQwOFwwNDBUZlwwNDAwXDA0MDBcMDQwMFwwNTY1MDJcMDQwcmcpCi9GIDQKL0ZUIC9UeAovRmYgODM4ODYwOQovTUsgPDwKPj4KL1AgNCAwIFIKL1EgMAovUmVjdCBbIDM2IDQ5Mi4wMDYgNDM5LjIgNTA2LjAwNyBdCi9TdHJ1Y3RQYXJlbnQgNAovU3VidHlwZSAvV2lkZ2V0Ci9UIChDaXR5U3RhdGUpCi9UeXBlIC9Bbm5vdAovViAoTG9uZG9uXDA0MEhpbGxpbmdkb25cMDQwVUI3XDA0MDlBUSkKL0FQIDw8Ci9OIDgwIDAgUgo-Pgo-PgplbmRvYmoKNDUgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgNDQxLjIgNDkyLjAwNiA1NzYgNTA2LjAwNyBdCi9TdHJ1Y3RQYXJlbnQgNQovU3VidHlwZSAvV2lkZ2V0Ci9UIChDb3VudHJ5KQovVHlwZSAvQW5ub3QKL1YgKFVuaXRlZFwwNDBLaW5nZG9tXDA0MFwwNTBFbmdsYW5kXDA1NFwwNDBOb3J0aGVyblwwNDBJcmVsYW5kXDA1NFwwNDBTY290bGFuZFwwNTRcMDQwYW5kXDA0MFdhbGVzXDA1MSkKL0FQIDw8Ci9OIDgxIDAgUgo-Pgo-PgplbmRvYmoKNDYgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgMzYgNDY4LjAwOCA1NzYgNDgyLjAwOSBdCi9TdHJ1Y3RQYXJlbnQgNgovU3VidHlwZSAvV2lkZ2V0Ci9UIChNYWlsaW5nQWRkcmVzcykKL1R5cGUgL0Fubm90Cj4-CmVuZG9iago0NyAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL0ZmIDgzODg2MDkKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDAKL1JlY3QgWyAzNiA0NDQuMDA3IDQzOS4yIDQ1OC4wMDggXQovU3RydWN0UGFyZW50IDcKL1N1YnR5cGUgL1dpZGdldAovVCAoTWFpbGluZ0NpdHlTdGF0ZSkKL1R5cGUgL0Fubm90Ci9WIChcMDQwXDA0MCkKL0FQIDw8Ci9OIDgyIDAgUgo-Pgo-PgplbmRvYmoKNDggMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgNDQxLjIgNDQ0LjAwNyA1NzYgNDU4LjAwOCBdCi9TdHJ1Y3RQYXJlbnQgOAovU3VidHlwZSAvV2lkZ2V0Ci9UIChNYWlsaW5nQ291bnRyeSkKL1R5cGUgL0Fubm90Cj4-CmVuZG9iago0OSAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDEKL1JlY3QgWyAzNi40MjAzIDQyMC41MzUgNTc2IDQzNC4xMSBdCi9TdHJ1Y3RQYXJlbnQgOQovU3VidHlwZSAvV2lkZ2V0Ci9UIChTU05vcklUSU4pCi9UeXBlIC9Bbm5vdAovViAoKQovQVAgPDwKL04gODMgMCBSCj4-Ci9GZiAxCj4-CmVuZG9iago1MCAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL0ZmIDgzODg2MDkKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDEKL1JlY3QgWyAzNiAzOTYuMDA4IDI4OCA0MDguMDE4IF0KL1N0cnVjdFBhcmVudCAxMAovU3VidHlwZSAvV2lkZ2V0Ci9UIChGb3JlaWduVElOKQovVHlwZSAvQW5ub3QKL1YgKFNDNTMyNDY0QSkKL0FQIDw8Ci9OIDg0IDAgUgo-Pgo-PgplbmRvYmoKNTEgMCBvYmoKPDwKL0FQIDw8Ci9EIDw8Ci8xIDUyIDAgUgovT2ZmIDU0IDAgUgo-PgovTiA8PAovMSA1NSAwIFIKPj4KPj4KL0FTIC9PZmYKL0RBIChcMDU3WmFEYlwwNDAxMlwwNDBUZlwwNDAwXDA0MGcpCi9GIDQKL0ZUIC9CdG4KL01LIDw8Ci9DQSAoNCkKPj4KL1AgNCAwIFIKL1JlY3QgWyA1NjMuNTk5IDQxMC4wMSA1NzEuNTk5IDQxOC4wMSBdCi9TdHJ1Y3RQYXJlbnQgMTEKL1N1YnR5cGUgL1dpZGdldAovVCAoRm9yZWlnblRJTlJlcXVpcmVkKQovVHlwZSAvQW5ub3QKL1YgKFwwNTdPZmYpCi9GZiAxCj4-CmVuZG9iago1MiAwIG9iago8PAovQkJveCBbIDAuMCAwLjAgOCA4IF0KL0Zvcm1UeXBlIDEKL01hdHJpeCBbIDEgMC4wIDAuMCAxIDAuMCAwLjAgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9aYURiIDUzIDAgUgo-PgovUHJvY1NldCBbIC9QREYgL1RleHQgXQo-PgovU3VidHlwZSAvRm9ybQovVHlwZSAvWE9iamVjdAovTGVuZ3RoIDEwMAo-PgpzdHJlYW0KcQowLjc0OTAyMyBnCjAgMCA4IDggcmUKZgpRCnEKMSAxIDYgNiByZQpXCm4KQlQKL1phRGIgMTIgVGYKLTEuMDc2IC0wLjA2MTkgVGQKMTEuNTU2IFRMCig0KSBUagpFVApRCgplbmRzdHJlYW0KZW5kb2JqCjUzIDAgb2JqCjw8Ci9CYXNlRm9udCAvWmFwZkRpbmdiYXRzCi9TdWJ0eXBlIC9UeXBlMQovVHlwZSAvRm9udAo-PgplbmRvYmoKNTQgMCBvYmoKPDwKL0JCb3ggWyAwLjAgMC4wIDggOCBdCi9Gb3JtVHlwZSAxCi9NYXRyaXggWyAxIDAuMCAwLjAgMSAwLjAgMC4wIF0KL1Jlc291cmNlcyA8PAovUHJvY1NldCBbIC9QREYgXQo-PgovU3VidHlwZSAvRm9ybQovVHlwZSAvWE9iamVjdAovTGVuZ3RoIDI4Cj4-CnN0cmVhbQpxCjAuNzQ5MDIzIGcKMCAwIDggOCByZQpmClEKCmVuZHN0cmVhbQplbmRvYmoKNTUgMCBvYmoKPDwKL0JCb3ggWyAwLjAgMC4wIDggOCBdCi9Gb3JtVHlwZSAxCi9NYXRyaXggWyAxIDAuMCAwLjAgMSAwLjAgMC4wIF0KL1Jlc291cmNlcyA8PAovRm9udCA8PAovWmFEYiA1MyAwIFIKPj4KL1Byb2NTZXQgWyAvUERGIC9UZXh0IF0KPj4KL1N1YnR5cGUgL0Zvcm0KL1R5cGUgL1hPYmplY3QKL0xlbmd0aCA3Mgo-PgpzdHJlYW0KcQoxIDEgNiA2IHJlClcKbgpCVAovWmFEYiAxMiBUZgotMS4wNzYgLTAuMDYxOSBUZAoxMS41NTYgVEwKKDQpIFRqCkVUClEKCmVuZHN0cmVhbQplbmRvYmoKNTYgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAxCi9SZWN0IFsgMzYgMzcyLjAwNyAyODggMzgzLjY3MSBdCi9TdHJ1Y3RQYXJlbnQgMTIKL1N1YnR5cGUgL1dpZGdldAovVCAoUmVmZXJlbmNlTnVtYmVycykKL1R5cGUgL0Fubm90Ci9WIChcMDQwKQovQVAgPDwKL04gODUgMCBSCj4-Cj4-CmVuZG9iago1NyAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDEKL1JlY3QgWyAyODggMzcyLjAwNyA1NzYgMzgzLjk4OSBdCi9TdHJ1Y3RQYXJlbnQgMTMKL1N1YnR5cGUgL1dpZGdldAovVCAoRGF0ZU9mQmlydGgpCi9UeXBlIC9Bbm5vdAovViAoMTBcMDU1MThcMDU1MTk5NykKL0FQIDw8Ci9OIDg2IDAgUgo-PgovRmYgMQo-PgplbmRvYmoKNTggMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgMjM3LjYgMzQ4LjAwOSA0MzkuMiAzNjAuMDA4IF0KL1N0cnVjdFBhcmVudCAxNAovU3VidHlwZSAvV2lkZ2V0Ci9UIChDb3VudHJ5T2ZSZXNpZGVuY2UpCi9UeXBlIC9Bbm5vdAovViAoVW5pdGVkXDA0MEtpbmdkb21cMDQwXDA1MEVuZ2xhbmRcMDU0XDA0ME5vcnRoZXJuXDA0MElyZWxhbmRcMDU0XDA0MFNjb3RsYW5kXDA1NFwwNDBhbmRcMDQwV2FsZXNcMDUxKQovQVAgPDwKL04gODcgMCBSCj4-Cj4-CmVuZG9iago1OSAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL0ZmIDgzODg2MDkKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDAKL1JlY3QgWyA2NC44IDMxMi4wMDkgMTc2LjI5OCAzMjQuMDA4IF0KL1N0cnVjdFBhcmVudCAxNQovU3VidHlwZSAvV2lkZ2V0Ci9UIChDbGFpbUFydGljbGVBbmRQYXJhZ3JhcGgpCi9UeXBlIC9Bbm5vdAo-PgplbmRvYmoKNjAgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAyCi9SZWN0IFsgMzUyLjQ4NCAzMTIuMDA5IDM3NC40IDMyNC4wMDggXQovU3RydWN0UGFyZW50IDE2Ci9TdWJ0eXBlIC9XaWRnZXQKL1QgKFdpdGhob2xkaW5nUmF0ZSkKL1R5cGUgL0Fubm90Cj4-CmVuZG9iago2MSAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDhcMDQwVGZcMDQwMFwwNDAwXDA0MDBcMDU2NTAyXDA0MHJnKQovRiA0Ci9GVCAvVHgKL0ZmIDgzODg2MDkKL01LIDw8Cj4-Ci9QIDQgMCBSCi9RIDAKL1JlY3QgWyA2NC44IDMwMC4wMSA1NjguOCAzMTIuMDA5IF0KL1N0cnVjdFBhcmVudCAxNwovU3VidHlwZSAvV2lkZ2V0Ci9UIChJbmNvbWVUeXBlKQovVHlwZSAvQW5ub3QKPj4KZW5kb2JqCjYyIDAgb2JqCjw8Ci9EQSAoXDA1N0NvdXJcMDQwOFwwNDBUZlwwNDAwXDA0MDBcMDQwMFwwNTY1MDJcMDQwcmcpCi9GIDQKL0ZUIC9UeAovRmYgODM4ODYwOQovTUsgPDwKPj4KL1AgNCAwIFIKL1EgMAovUmVjdCBbIDY0LjggMjc2LjAwOSA1NzYgMjg4LjAwOCBdCi9TdHJ1Y3RQYXJlbnQgMTkKL1N1YnR5cGUgL1dpZGdldAovVCAoRXhwbGFuYXRpb25PZkVsaWdpYmlsaXR5KQovVHlwZSAvQW5ub3QKPj4KZW5kb2JqCjYzIDAgb2JqCjw8Ci9BUCA8PAovRCA8PAovMSA2NCAwIFIKL09mZiA2NSAwIFIKPj4KL04gPDwKLzEgNjYgMCBSCj4-Cj4-Ci9BUyAvMQovREEgKFwwNTdaYURiXDA0MDEyXDA0MFRmXDA0MDBcMDQwZykKL0YgNAovRlQgL0J0bgovTUsgPDwKL0NBICg0KQo-PgovUCA0IDAgUgovUmVjdCBbIDEwOS45OTkgOTguMDA5IDExNy45OTkgMTA2LjAwOSBdCi9TdHJ1Y3RQYXJlbnQgMjAKL1N1YnR5cGUgL1dpZGdldAovVCAoQ2VydGlmeUNhblNpZ24pCi9UeXBlIC9Bbm5vdAovViAoXDA1NzEpCi9GZiAxCj4-CmVuZG9iago2NCAwIG9iago8PAovQkJveCBbIDAuMCAwLjAgOCA4IF0KL0Zvcm1UeXBlIDEKL01hdHJpeCBbIDEgMC4wIDAuMCAxIDAuMCAwLjAgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9aYURiIDUzIDAgUgo-PgovUHJvY1NldCBbIC9QREYgL1RleHQgXQo-PgovU3VidHlwZSAvRm9ybQovVHlwZSAvWE9iamVjdAovTGVuZ3RoIDEwMAo-PgpzdHJlYW0KcQowLjc0OTAyMyBnCjAgMCA4IDggcmUKZgpRCnEKMSAxIDYgNiByZQpXCm4KQlQKL1phRGIgMTIgVGYKLTEuMDc2IC0wLjA2MTkgVGQKMTEuNTU2IFRMCig0KSBUagpFVApRCgplbmRzdHJlYW0KZW5kb2JqCjY1IDAgb2JqCjw8Ci9CQm94IFsgMC4wIDAuMCA4IDggXQovRm9ybVR5cGUgMQovTWF0cml4IFsgMSAwLjAgMC4wIDEgMC4wIDAuMCBdCi9SZXNvdXJjZXMgPDwKL1Byb2NTZXQgWyAvUERGIF0KPj4KL1N1YnR5cGUgL0Zvcm0KL1R5cGUgL1hPYmplY3QKL0xlbmd0aCAyOAo-PgpzdHJlYW0KcQowLjc0OTAyMyBnCjAgMCA4IDggcmUKZgpRCgplbmRzdHJlYW0KZW5kb2JqCjY2IDAgb2JqCjw8Ci9CQm94IFsgMC4wIDAuMCA4IDggXQovRm9ybVR5cGUgMQovTWF0cml4IFsgMSAwLjAgMC4wIDEgMC4wIDAuMCBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL1phRGIgNTMgMCBSCj4-Ci9Qcm9jU2V0IFsgL1BERiAvVGV4dCBdCj4-Ci9TdWJ0eXBlIC9Gb3JtCi9UeXBlIC9YT2JqZWN0Ci9MZW5ndGggNzIKPj4Kc3RyZWFtCnEKMSAxIDYgNiByZQpXCm4KQlQKL1phRGIgMTIgVGYKLTEuMDc2IC0wLjA2MTkgVGQKMTEuNTU2IFRMCig0KSBUagpFVApRCgplbmRzdHJlYW0KZW5kb2JqCjY3IDAgb2JqCjw8Ci9EQSAoXDA1N0NvdXJcMDQwMTJcMDQwVGZcMDQwMFwwNDBnKQovRiA0Ci9GVCAvVHgKL01LIDw8Cj4-Ci9QIDQgMCBSCi9SZWN0IFsgMTA3LjM0NiA3Ni45NjI2IDQyNS41NjUgOTguOTYzIF0KL1N1YnR5cGUgL1dpZGdldAovVCAoQ2VydGlmaWNhdGlvblNpZ25hdHVyZSkKL1R5cGUgL0Fubm90Ci9WIChNaWNoYWVsXDA0MEFsaSkKL0FQIDw8Ci9OIDg4IDAgUgo-PgovRmYgMQo-PgplbmRvYmoKNjggMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAxCi9SZWN0IFsgNDMyIDcyLjAwNiA1NzYgODQuMDA3IF0KL1N0cnVjdFBhcmVudCAyMgovU3VidHlwZSAvV2lkZ2V0Ci9UIChEYXRlT2ZTaWduYXR1cmUpCi9UeXBlIC9Bbm5vdAovViAoMTJcMDU1MjFcMDU1MjAyNCkKL0FQIDw8Ci9OIDg5IDAgUgo-PgovRmYgMQo-PgplbmRvYmoKNjkgMCBvYmoKPDwKL0RBIChcMDU3Q291clwwNDA4XDA0MFRmXDA0MDBcMDQwMFwwNDAwXDA1NjUwMlwwNDByZykKL0YgNAovRlQgL1R4Ci9GZiA4Mzg4NjA5Ci9NSyA8PAo-PgovUCA0IDAgUgovUSAwCi9SZWN0IFsgMTA4IDQ4LjAwNyA0MjQuOCA2MC4wMDkgXQovU3RydWN0UGFyZW50IDIzCi9TdWJ0eXBlIC9XaWRnZXQKL1QgKFByaW50TmFtZU9mU2lnbmF0dXJlKQovVHlwZSAvQW5ub3QKL1YgKE1pY2hhZWxcMDQwQWxpKQovQVAgPDwKL04gOTAgMCBSCj4-Cj4-CmVuZG9iago3MCAwIG9iago8PAovREEgKFwwNTdDb3VyXDA0MDZcMDQwVGZcMDQwMFwwNDAwXDA0MDFcMDQwcmcpCi9GIDQKL0ZUIC9UeAovRmYgMTI1ODI5MTMKL01LIDw8Cj4-Ci9QIDQgMCBSCi9SZWN0IFsgMTA3LjIxMiA3MS4wMjc0IDQyNS45MTMgODEuNDY1MyBdCi9TdWJ0eXBlIC9XaWRnZXQKL1QgKERpZ2l0YWxTaWduYXR1cmUpCi9UeXBlIC9Bbm5vdAovViAoRGlnaXRhbGx5XDA0MHNpZ25lZFwwNDBieVwwNDBNSUNIQUVMXDA0MEFMSVwwNDBvblwwNDAyMDI0XDA1NTEyXDA1NTIxXDA0MDIzXDA3MjExXDA3MjIxXDA0MFwwNTMwMFwwNzIwMCkKL0FQIDw8Ci9OIDkxIDAgUgo-Pgo-PgplbmRvYmoKNzEgMCBvYmoKPDwKL0RBIChcMDU3SGVsdlwwNDAwXDA0MFRmXDA0MDBcMDQwZ1wwNDApCi9EUiA8PAovRW5jb2RpbmcgPDwKL1BERkRvY0VuY29kaW5nIDcyIDAgUgo-PgovRm9udCA8PAovQ291ciA3MyAwIFIKL0hlbHYgNzQgMCBSCi9IZWx2ZXRpY2FMVFN0ZC1Cb2xkIDc1IDAgUgovWmFEYiA1MyAwIFIKPj4KPj4KL1NpZ0ZsYWdzIDIKL0ZpZWxkcyBbIDQxIDAgUiA0MiAwIFIgNDMgMCBSIDQ0IDAgUiA0NSAwIFIgNDYgMCBSIDQ3IDAgUiA0OCAwIFIgNDkgMCBSIDUwIDAgUiA1MSAwIFIgNTYgMCBSIDU3IDAgUiA1OCAwIFIgNTkgMCBSIDYwIDAgUiA2MSAwIFIgNjIgMCBSIDYzIDAgUiA2OCAwIFIgNjkgMCBSIDY3IDAgUiA3MCAwIFIgXQovTmVlZEFwcGVhcmFuY2VzIGZhbHNlCj4-CmVuZG9iago3MiAwIG9iago8PAovRGlmZmVyZW5jZXMgWyAyNCAvYnJldmUgL2Nhcm9uIC9jaXJjdW1mbGV4IC9kb3RhY2NlbnQgL2h1bmdhcnVtbGF1dCAvb2dvbmVrIC9yaW5nIC90aWxkZSAzOSAvcXVvdGVzaW5nbGUgOTYgL2dyYXZlIDEyOCAvYnVsbGV0IC9kYWdnZXIgL2RhZ2dlcmRibCAvZWxsaXBzaXMgL2VtZGFzaCAvZW5kYXNoIC9mbG9yaW4gL2ZyYWN0aW9uIC9ndWlsc2luZ2xsZWZ0IC9ndWlsc2luZ2xyaWdodCAvbWludXMgL3BlcnRob3VzYW5kIC9xdW90ZWRibGJhc2UgL3F1b3RlZGJsbGVmdCAvcXVvdGVkYmxyaWdodCAvcXVvdGVsZWZ0IC9xdW90ZXJpZ2h0IC9xdW90ZXNpbmdsYmFzZSAvdHJhZGVtYXJrIC9maSAvZmwgL0xzbGFzaCAvT0UgL1NjYXJvbiAvWWRpZXJlc2lzIC9aY2Fyb24gL2RvdGxlc3NpIC9sc2xhc2ggL29lIC9zY2Fyb24gL3pjYXJvbiAxNjAgL0V1cm8gMTY0IC9jdXJyZW5jeSAxNjYgL2Jyb2tlbmJhciAxNjggL2RpZXJlc2lzIC9jb3B5cmlnaHQgL29yZGZlbWluaW5lIDE3MiAvbG9naWNhbG5vdCAvLm5vdGRlZiAvcmVnaXN0ZXJlZCAvbWFjcm9uIC9kZWdyZWUgL3BsdXNtaW51cyAvdHdvc3VwZXJpb3IgL3RocmVlc3VwZXJpb3IgL2FjdXRlIC9tdSAxODMgL3BlcmlvZGNlbnRlcmVkIC9jZWRpbGxhIC9vbmVzdXBlcmlvciAvb3JkbWFzY3VsaW5lIDE4OCAvb25lcXVhcnRlciAvb25laGFsZiAvdGhyZWVxdWFydGVycyAxOTIgL0FncmF2ZSAvQWFjdXRlIC9BY2lyY3VtZmxleCAvQXRpbGRlIC9BZGllcmVzaXMgL0FyaW5nIC9BRSAvQ2NlZGlsbGEgL0VncmF2ZSAvRWFjdXRlIC9FY2lyY3VtZmxleCAvRWRpZXJlc2lzIC9JZ3JhdmUgL0lhY3V0ZSAvSWNpcmN1bWZsZXggL0lkaWVyZXNpcyAvRXRoIC9OdGlsZGUgL09ncmF2ZSAvT2FjdXRlIC9PY2lyY3VtZmxleCAvT3RpbGRlIC9PZGllcmVzaXMgL211bHRpcGx5IC9Pc2xhc2ggL1VncmF2ZSAvVWFjdXRlIC9VY2lyY3VtZmxleCAvVWRpZXJlc2lzIC9ZYWN1dGUgL1Rob3JuIC9nZXJtYW5kYmxzIC9hZ3JhdmUgL2FhY3V0ZSAvYWNpcmN1bWZsZXggL2F0aWxkZSAvYWRpZXJlc2lzIC9hcmluZyAvYWUgL2NjZWRpbGxhIC9lZ3JhdmUgL2VhY3V0ZSAvZWNpcmN1bWZsZXggL2VkaWVyZXNpcyAvaWdyYXZlIC9pYWN1dGUgL2ljaXJjdW1mbGV4IC9pZGllcmVzaXMgL2V0aCAvbnRpbGRlIC9vZ3JhdmUgL29hY3V0ZSAvb2NpcmN1bWZsZXggL290aWxkZSAvb2RpZXJlc2lzIC9kaXZpZGUgL29zbGFzaCAvdWdyYXZlIC91YWN1dGUgL3VjaXJjdW1mbGV4IC91ZGllcmVzaXMgL3lhY3V0ZSAvdGhvcm4gL3lkaWVyZXNpcyBdCi9UeXBlIC9FbmNvZGluZwo-PgplbmRvYmoKNzMgMCBvYmoKPDwKL0Jhc2VGb250IC9Db3VyaWVyCi9FbmNvZGluZyA3MiAwIFIKL05hbWUgL0NvdXIKL1N1YnR5cGUgL1R5cGUxCi9UeXBlIC9Gb250Cj4-CmVuZG9iago3NCAwIG9iago8PAovQmFzZUZvbnQgL0hlbHZldGljYQovRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZwovU3VidHlwZSAvVHlwZTEKL1R5cGUgL0ZvbnQKPj4KZW5kb2JqCjc1IDAgb2JqCjw8Ci9CYXNlRm9udCAvSGVsdmV0aWNhTFRTdGQtQm9sZAovRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZwovRmlyc3RDaGFyIDAKL0ZvbnREZXNjcmlwdG9yIDc2IDAgUgovTGFzdENoYXIgMjU1Ci9TdWJ0eXBlIC9UeXBlMQovVHlwZSAvRm9udAovV2lkdGhzIFsgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCA1MDAgNTAwIDUwMCAyNzggMzMzIDQ3NCA1NTYgNTU2IDg4OSA3MjIgMjM4IDMzMyAzMzMgMzg5IDU4NCAyNzggMzMzIDI3OCAyNzggNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiA1NTYgNTU2IDMzMyAzMzMgNTg0IDU4NCA1ODQgNjExIDk3NSA3MjIgNzIyIDcyMiA3MjIgNjY3IDYxMSA3NzggNzIyIDI3OCA1NTYgNzIyIDYxMSA4MzMgNzIyIDc3OCA2NjcgNzc4IDcyMiA2NjcgNjExIDcyMiA2NjcgOTQ0IDY2NyA2NjcgNjExIDMzMyAyNzggMzMzIDU4NCA1NTYgMzMzIDU1NiA2MTEgNTU2IDYxMSA1NTYgMzMzIDYxMSA2MTEgMjc4IDI3OCA1NTYgMjc4IDg4OSA2MTEgNjExIDYxMSA2MTEgMzg5IDU1NiAzMzMgNjExIDU1NiA3NzggNTU2IDU1NiA1MDAgMzg5IDI4MCAzODkgNTg0IDM1MCA1NTYgMzUwIDI3OCA1NTYgNTAwIDEwMDAgNTU2IDU1NiAzMzMgMTAwMCA2NjcgMzMzIDEwMDAgMzUwIDYxMSAzNTAgMzUwIDI3OCAyNzggNTAwIDUwMCAzNTAgNTU2IDEwMDAgMzMzIDEwMDAgNTU2IDMzMyA5NDQgMzUwIDUwMCA2NjcgMjc4IDMzMyA1NTYgNTU2IDU1NiA1NTYgMjgwIDU1NiAzMzMgNzM3IDM3MCA1NTYgNTg0IDMzMyA3MzcgMzMzIDQwMCA1ODQgNTAwIDUwMCAzMzMgNjExIDU1NiAyNzggMzMzIDUwMCAzNjUgNTU2IDgzNCA4MzQgODM0IDYxMSA3MjIgNzIyIDcyMiA3MjIgNzIyIDcyMiAxMDAwIDcyMiA2NjcgNjY3IDY2NyA2NjcgMjc4IDI3OCAyNzggMjc4IDcyMiA3MjIgNzc4IDc3OCA3NzggNzc4IDc3OCA1ODQgNzc4IDcyMiA3MjIgNzIyIDcyMiA2NjcgNjY3IDYxMSA1NTYgNTU2IDU1NiA1NTYgNTU2IDU1NiA4ODkgNTU2IDU1NiA1NTYgNTU2IDU1NiAyNzggMjc4IDI3OCAyNzggNjExIDYxMSA2MTEgNjExIDYxMSA2MTEgNjExIDU4NCA2MTEgNjExIDYxMSA2MTEgNjExIDU1NiA2MTEgNTU2IF0KPj4KZW5kb2JqCjc2IDAgb2JqCjw8Ci9Bc2NlbnQgOTYyCi9DYXBIZWlnaHQgNzE4Ci9EZXNjZW50IC0yMjgKL0ZsYWdzIDMyCi9Gb250QkJveCBbIC0xNzAgLTIyOCAxMDAzIDk2MiBdCi9Gb250RmFtaWx5IChIZWx2ZXRpY2FcMDQwTFRcMDQwU3RkKQovRm9udE5hbWUgL0hlbHZldGljYUxUU3RkLUJvbGQKL0ZvbnRTdHJldGNoIC9Ob3JtYWwKL0ZvbnRXZWlnaHQgNzAwCi9JdGFsaWNBbmdsZSAwCi9TdGVtViAxNDAKL1R5cGUgL0ZvbnREZXNjcmlwdG9yCi9YSGVpZ2h0IDUzMgo-PgplbmRvYmoKNzcgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDMzOC40IDE0LjAwMSBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL0NvdXIgNzMgMCBSCj4-Cj4-Ci9MZW5ndGggMTI1Cj4-CnN0cmVhbQpxCi9UeCBCTUMgCnEKMSAxIDMzNy40IDEzLjAwMDk5OTk5OTk5OTk3NiByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgNS4wMDA5OTk5OTk5OTk5NzYgVGQKKE1JQ0hBRUwgQUxJKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCjc4IDAgb2JqCjw8Ci9UeXBlIC9YT2JqZWN0Ci9TdWJ0eXBlIC9Gb3JtCi9CQm94IFsgMC4wIDAuMCAxOTkuNiAxNC4wMDEgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDE5Mgo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAxOTguNjAwMDAwMDAwMDAwMDIgMTMuMDAwOTk5OTk5OTk5OTc2IHJlClcKQlQKL0NvdXIgOCBUZiAwIDAgMC41MDIgcmcKMiA1LjAwMDk5OTk5OTk5OTk3NiBUZAooVW5pdGVkIEtpbmdkb20gXChFbmdsYW5kLCBOb3J0aGVybiBJcmVsYW5kLCBTY290bGFuZCwgYW5kIFdhbGVzXCkpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKNzkgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDU0MCAxNC4wMDEgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDEzMgo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSA1MzkuMCAxMy4wMDEwMDAwMDAwMDAwOSByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgNS4wMDEwMDAwMDAwMDAwOSBUZAooOTMgTXVsYmVycnkgQ3Jlc2NlbnQpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKODAgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDQwMy4yIDE0LjAwMSBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL0NvdXIgNzMgMCBSCj4-Cj4-Ci9MZW5ndGggMTM5Cj4-CnN0cmVhbQpxCi9UeCBCTUMgCnEKMSAxIDQwMi4yIDEzLjAwMTAwMDAwMDAwMDAzMyByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgNS4wMDEwMDAwMDAwMDAwMzMgVGQKKExvbmRvbiBIaWxsaW5nZG9uIFVCNyA5QVEpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKODEgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDEzNC44IDE0LjAwMSBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL0NvdXIgNzMgMCBSCj4-Cj4-Ci9MZW5ndGggMTc5Cj4-CnN0cmVhbQpxCi9UeCBCTUMgCnEKMSAxIDEzMy44IDEzLjAwMTAwMDAwMDAwMDAzMyByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgNS4wMDEwMDAwMDAwMDAwMzMgVGQKKFVuaXRlZCBLaW5nZG9tIFwoRW5nbGFuZCwgTm9ydGhlcm4gSXJlbGFuZCwgU2NvdGxhbmQsIGFuZCBXYWxlc1wpKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCjgyIDAgb2JqCjw8Ci9UeXBlIC9YT2JqZWN0Ci9TdWJ0eXBlIC9Gb3JtCi9CQm94IFsgMC4wIDAuMCA0MDMuMiAxNC4wMDEgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDExNgo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSA0MDIuMiAxMy4wMDA5OTk5OTk5OTk5NzYgcmUKVwpCVAovQ291ciA4IFRmIDAgMCAwLjUwMiByZwoyIDUuMDAwOTk5OTk5OTk5OTc2IFRkCiggICkgVGoKRVQKUQpFTUMKUQoKZW5kc3RyZWFtCmVuZG9iago4MyAwIG9iago8PAovVHlwZSAvWE9iamVjdAovU3VidHlwZSAvRm9ybQovQkJveCBbIDAuMCAwLjAgNTM5LjU3OTcgMTMuNTc1IF0KL1Jlc291cmNlcyA8PAovRm9udCA8PAovQ291ciA3MyAwIFIKPj4KPj4KL0xlbmd0aCAxMTcKPj4Kc3RyZWFtCnEKL1R4IEJNQyAKcQoxIDEgNTM4LjU3OTcgMTIuNTc0OTk5OTk5OTk5OTg5IHJlClcKQlQKL0NvdXIgOCBUZiAwIDAgMC41MDIgcmcKMiA0LjU3NDk5OTk5OTk5OTk4OSBUZAooKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCjg0IDAgb2JqCjw8Ci9UeXBlIC9YT2JqZWN0Ci9TdWJ0eXBlIC9Gb3JtCi9CQm94IFsgMC4wIDAuMCAyNTIgMTIuMDEgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDEyMwo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAyNTEuMCAxMS4wMDk5OTk5OTk5OTk5OTEgcmUKVwpCVAovQ291ciA4IFRmIDAgMCAwLjUwMiByZwoyIDMuMDA5OTk5OTk5OTk5OTkxIFRkCihTQzUzMjQ2NEEpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKODUgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDI1MiAxMS42NjQgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDExNgo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAyNTEuMCAxMC42NjM5OTk5OTk5OTk5ODcgcmUKVwpCVAovQ291ciA4IFRmIDAgMCAwLjUwMiByZwoyIDIuNjYzOTk5OTk5OTk5OTg3MyBUZAooICkgVGoKRVQKUQpFTUMKUQoKZW5kc3RyZWFtCmVuZG9iago4NiAwIG9iago8PAovVHlwZSAvWE9iamVjdAovU3VidHlwZSAvRm9ybQovQkJveCBbIDAuMCAwLjAgMjg4IDExLjk4MiBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL0NvdXIgNzMgMCBSCj4-Cj4-Ci9MZW5ndGggMTI0Cj4-CnN0cmVhbQpxCi9UeCBCTUMgCnEKMSAxIDI4Ny4wIDEwLjk4MTk5OTk5OTk5OTk3MSByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgMi45ODE5OTk5OTk5OTk5NzEgVGQKKDEwLTE4LTE5OTcpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKODcgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDIwMS42IDExLjk5OSBdCi9SZXNvdXJjZXMgPDwKL0ZvbnQgPDwKL0NvdXIgNzMgMCBSCj4-Cj4-Ci9MZW5ndGggMTc5Cj4-CnN0cmVhbQpxCi9UeCBCTUMgCnEKMSAxIDIwMC42IDEwLjk5ODk5OTk5OTk5OTk2NyByZQpXCkJUCi9Db3VyIDggVGYgMCAwIDAuNTAyIHJnCjIgMi45OTg5OTk5OTk5OTk5NjcgVGQKKFVuaXRlZCBLaW5nZG9tIFwoRW5nbGFuZCwgTm9ydGhlcm4gSXJlbGFuZCwgU2NvdGxhbmQsIGFuZCBXYWxlc1wpKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCjg4IDAgb2JqCjw8Ci9UeXBlIC9YT2JqZWN0Ci9TdWJ0eXBlIC9Gb3JtCi9CQm94IFsgMC4wIDAuMCAzMTguMjE5IDIyLjAwMDQgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDEwOAo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAzMTcuMjE5IDIxLjAwMDQgcmUKVwpCVAovQ291ciAxMiBUZiAwIGcKMiA5LjAwMDM5OTk5OTk5OTk5OSBUZAooTWljaGFlbCBBbGkpIFRqCkVUClEKRU1DClEKCmVuZHN0cmVhbQplbmRvYmoKODkgMCBvYmoKPDwKL1R5cGUgL1hPYmplY3QKL1N1YnR5cGUgL0Zvcm0KL0JCb3ggWyAwLjAgMC4wIDE0NCAxMi4wMDEgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDEyNQo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAxNDMuMCAxMS4wMDEwMDAwMDAwMDAwMDUgcmUKVwpCVAovQ291ciA4IFRmIDAgMCAwLjUwMiByZwoyIDMuMDAxMDAwMDAwMDAwMDA0OCBUZAooMTItMjEtMjAyNCkgVGoKRVQKUQpFTUMKUQoKZW5kc3RyZWFtCmVuZG9iago5MCAwIG9iago8PAovVHlwZSAvWE9iamVjdAovU3VidHlwZSAvRm9ybQovQkJveCBbIDAuMCAwLjAgMzE2LjggMTIuMDAyIF0KL1Jlc291cmNlcyA8PAovRm9udCA8PAovQ291ciA3MyAwIFIKPj4KPj4KL0xlbmd0aCAxMjYKPj4Kc3RyZWFtCnEKL1R4IEJNQyAKcQoxIDEgMzE1LjggMTEuMDAyMDAwMDAwMDAwMDAyIHJlClcKQlQKL0NvdXIgOCBUZiAwIDAgMC41MDIgcmcKMiAzLjAwMjAwMDAwMDAwMDAwMjQgVGQKKE1pY2hhZWwgQWxpKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCjkxIDAgb2JqCjw8Ci9UeXBlIC9YT2JqZWN0Ci9TdWJ0eXBlIC9Gb3JtCi9CQm94IFsgMC4wIDAuMCAzMTguNzAxIDEwLjQzNzkgXQovUmVzb3VyY2VzIDw8Ci9Gb250IDw8Ci9Db3VyIDczIDAgUgo-Pgo-PgovTGVuZ3RoIDE3Mgo-PgpzdHJlYW0KcQovVHggQk1DIApxCjEgMSAzMTcuNzAxIDkuNDM3ODk5OTk5OTk5OTk5IHJlClcKQlQKL0NvdXIgNiBUZiAwIDAgMSByZwoyIDMuNDM3ODk5OTk5OTk5OTk5IFRkCihEaWdpdGFsbHkgc2lnbmVkIGJ5IE1JQ0hBRUwgQUxJIG9uIDIwMjQtMTItMjEgMjM6MTE6MjEgKzAwOjAwKSBUagpFVApRCkVNQwpRCgplbmRzdHJlYW0KZW5kb2JqCnhyZWYKMCA5MgowMDAwMDAwMDAwIDY1NTM1IGYgCjAwMDAwMDAwMTUgMDAwMDAgbiAKMDAwMDAwMDA3NCAwMDAwMCBuIAowMDAwMDAwMTEzIDAwMDAwIG4gCjAwMDAwMDAxOTYgMDAwMDAgbiAKMDAwMDAwMDY3NCAwMDAwMCBuIAowMDAwMDA1MDcyIDAwMDAwIG4gCjAwMDAwMDUyMzAgMDAwMDAgbiAKMDAwMDAwNTI1NSAwMDAwMCBuIAowMDAwMDA1NDEwIDAwMDAwIG4gCjAwMDAwMDU0ODQgMDAwMDAgbiAKMDAwMDAwNTgyMCAwMDAwMCBuIAowMDAwMDA1OTA3IDAwMDAwIG4gCjAwMDAwMDY0NjYgMDAwMDAgbiAKMDAwMDAwNjc2OSAwMDAwMCBuIAowMDAwMDA2OTEyIDAwMDAwIG4gCjAwMDAwMDY5MzkgMDAwMDAgbiAKMDAwMDAwNzA5NyAwMDAwMCBuIAowMDAwMDA3MTcyIDAwMDAwIG4gCjAwMDAwMDc0NjYgMDAwMDAgbiAKMDAwMDAwNzU1MyAwMDAwMCBuIAowMDAwMDA4MDI1IDAwMDAwIG4gCjAwMDAwMDgzMjkgMDAwMDAgbiAKMDAwMDAwOTU0MCAwMDAwMCBuIAowMDAwMDEwMjkwIDAwMDAwIG4gCjAwMDAwMTQ1NzQgMDAwMDAgbiAKMDAwMDAxNTc4NCAwMDAwMCBuIAowMDAwMDE2MTUzIDAwMDAwIG4gCjAwMDAwMTcwOTQgMDAwMDAgbiAKMDAwMDAxODMwNSAwMDAwMCBuIAowMDAwMDE4ODE5IDAwMDAwIG4gCjAwMDAwMjE3MTggMDAwMDAgbiAKMDAwMDAyMjkyNyAwMDAwMCBuIAowMDAwMDIzNTcwIDAwMDAwIG4gCjAwMDAwMjczODMgMDAwMDAgbiAKMDAwMDAyODYxMCAwMDAwMCBuIAowMDAwMDI4OTUyIDAwMDAwIG4gCjAwMDAwMjk0MzYgMDAwMDAgbiAKMDAwMDAzMDY0NSAwMDAwMCBuIAowMDAwMDMxMDUzIDAwMDAwIG4gCjAwMDAwMzI3MDggMDAwMDAgbiAKMDAwMDAzMjczMCAwMDAwMCBuIAowMDAwMDMyOTg2IDAwMDAwIG4gCjAwMDAwMzMzNDQgMDAwMDAgbiAKMDAwMDAzMzYxMyAwMDAwMCBuIAowMDAwMDMzODk0IDAwMDAwIG4gCjAwMDAwMzQyMzkgMDAwMDAgbiAKMDAwMDAzNDQ2MyAwMDAwMCBuIAowMDAwMDM0NzI1IDAwMDAwIG4gCjAwMDAwMzQ5NTIgMDAwMDAgbiAKMDAwMDAzNTE5NSAwMDAwMCBuIAowMDAwMDM1NDUxIDAwMDAwIG4gCjAwMDAwMzU3NTAgMDAwMDAgbiAKMDAwMDAzNjA2NCAwMDAwMCBuIAowMDAwMDM2MTM4IDAwMDAwIG4gCjAwMDAwMzYzNDggMDAwMDAgbiAKMDAwMDAzNjYzMyAwMDAwMCBuIAowMDAwMDM2ODkwIDAwMDAwIG4gCjAwMDAwMzcxNDkgMDAwMDAgbiAKMDAwMDAzNzUwOCAwMDAwMCBuIAowMDAwMDM3NzQ5IDAwMDAwIG4gCjAwMDAwMzc5ODIgMDAwMDAgbiAKMDAwMDAzODIwNiAwMDAwMCBuIAowMDAwMDM4NDQzIDAwMDAwIG4gCjAwMDAwMzg3MzUgMDAwMDAgbiAKMDAwMDAzOTA0OSAwMDAwMCBuIAowMDAwMDM5MjU5IDAwMDAwIG4gCjAwMDAwMzk1NDQgMDAwMDAgbiAKMDAwMDAzOTc4MCAwMDAwMCBuIAowMDAwMDQwMDQxIDAwMDAwIG4gCjAwMDAwNDAzMTMgMDAwMDAgbiAKMDAwMDA0MDY1MCAwMDAwMCBuIAowMDAwMDQxMDQ4IDAwMDAwIG4gCjAwMDAwNDIzNzUgMDAwMDAgbiAKMDAwMDA0MjQ3MyAwMDAwMCBuIAowMDAwMDQyNTcxIDAwMDAwIG4gCjAwMDAwNDM3NzEgMDAwMDAgbiAKMDAwMDA0NDA0MyAwMDAwMCBuIAowMDAwMDQ0MzIzIDAwMDAwIG4gCjAwMDAwNDQ2NzAgMDAwMDAgbiAKMDAwMDA0NDk1NSAwMDAwMCBuIAowMDAwMDQ1MjQ5IDAwMDAwIG4gCjAwMDAwNDU1ODMgMDAwMDAgbiAKMDAwMDA0NTg1NCAwMDAwMCBuIAowMDAwMDQ2MTI5IDAwMDAwIG4gCjAwMDAwNDY0MDQgMDAwMDAgbiAKMDAwMDA0NjY3MyAwMDAwMCBuIAowMDAwMDQ2OTUwIDAwMDAwIG4gCjAwMDAwNDcyODQgMDAwMDAgbiAKMDAwMDA0NzU1MCAwMDAwMCBuIAowMDAwMDQ3ODI4IDAwMDAwIG4gCjAwMDAwNDgxMDkgMDAwMDAgbiAKdHJhaWxlcgo8PAovU2l6ZSA5MgovUm9vdCAzIDAgUgovSW5mbyAyIDAgUgovSUQgWyA8Mzc2MTYzMzY2MTMyMzU2NTYxMzAzNTM3MzczODMxNjI2NDMxNjQ2NDM3NjU2MjM2MzIzMjY0MzkzMzMwMzUzOD4gPDM3NjE2MzM2NjEzMjM1NjU2MTMwMzUzNzM3MzgzMTYyNjQzMTY0NjQzNzY1NjIzNjMyMzI2NDM5MzMzMDM1Mzg-IF0KPj4Kc3RhcnR4cmVmCjQ4NDM5CiUlRU9GCg=="

In [51]:
import base64

def decode_pdf_from_base64(base64_string, output_filename):
    """
    Decode a base64 string to a PDF file
    
    Args:
        base64_string (str): The base64 encoded string
        output_filename (str): Name of the output PDF file
    """
    # Remove potential base64 header if present
    if "base64," in base64_string:
        base64_string = base64_string.split("base64,")[1]
    
    # Decode the base64 string
    pdf_bytes = base64.urlsafe_b64decode(base64_string)
    
    # Write the bytes to a PDF file
    with open(output_filename, 'wb') as f:
        f.write(pdf_bytes)

decode_pdf_from_base64(base64_string, "decoded.pdf")

### Google Calendar

In [ ]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"Search for free slots for the next 14 days for id michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)

In [ ]:
result

In [ ]:
from services.composio import get_calendar_slots, find_free_slots

await get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv","outlook")


In [ ]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Checkup appointment on Friday, November 29th, 2024 at 2:30 PM'Create"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


In [4]:
from services.composio import book_appointment_composio
import logging

logger = logging.getLogger(__name__)


In [ ]:
from services.composio import book_appointment_composio

results = await book_appointment_composio("Checkup appointment on Friday, November 29th, 2024 at 2:30 PM", "user_2mmXezcGmjZCf88gT2v2waCBsXv")
results

In [ ]:
results